**Graduation Project :  Personalized Education Platform Powered by AI- Chatbot for Adaptive Learning**

**Team 1** - Sprints Bootcamp

**Team Members:**
  * Dina Fakhry
  * Abanoub Younan
  * Karen Emad
  * Basel Amr
  * Omar Tarek

**Date: 2024-03-24**

**Table of Content**

# 🚀 **2. Setting Up the Environment**

## **2.1 Connect to google colab**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# change directory
import os
os.chdir('/content/drive/MyDrive/Sprints task/Graduation Project')

## 2.2 Import Necessary libraries

In [3]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 18.1 MB/s eta 0:00:00


In [4]:
# 🔹 Standard Library Imports
import os
import re
import shutil
import mysql.connector
import sqlite3
import string
import time
import random
import warnings
from datetime import datetime, timedelta
from tqdm import tqdm
import sys

# 🔹 Data Science & Visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import graphviz

# 🔹 CLI & Table Formatting
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.theme import Theme
from rich.box import SIMPLE
from rich import box
from tabulate import tabulate
from prettytable import PrettyTable
from rich.table import Table as RichTable
from rich.text import Text


# 🔹 Initialize Rich Console
console = Console()

In [5]:
# to get packages initially installed in colab or in other word, the colab environment
!pip3 freeze > requirements.txt

# Connect to Databse

In [6]:
# Connection details
HOST = 'db.cl0oa2kwiw65.me-central-1.rds.amazonaws.com'
USER = 'admin'
PASSWORD = 'bo13QknuzQ5nKR95QU9i'
DB_NAME = 'CourseDataBase'

In [7]:
def connect_to_mysql(host, user, password, database=None):
    try:
        # Connect to the MySQL database
        db_conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        # Check if connection is successful
        if db_conn.is_connected():
            console.print(f"✅ Successfully connected to the MySQL database!", style="bold green")
            return db_conn
        else:
            console.print(f"❌ Failed to connect to the database.", style="bold red")
            return None
    except mysql.connector.Error as err:
        console.print(f"❌ Error: {err}", style="bold red")
        return None

In [8]:
db_conn = connect_to_mysql(
    host=HOST,
    user=USER,
    password=PASSWORD
)

✅ Successfully connected to the MySQL database!

In [9]:
cursor = db_conn.cursor()

In [10]:
# # 🚀 Drop the existing database (if it exists) and create a new one
# cursor.execute("DROP DATABASE IF EXISTS CourseDataBase;")
# print("⚠️ Database 'CourseDataBase' deleted.")

# cursor.execute("CREATE DATABASE CourseDataBase;")
# print("✅ Database 'CourseDataBase' created.")

# 🎯 Select the new database for further operations
cursor.execute("USE CourseDataBase;")
print("📌 Now using 'CourseDataBase'.")

📌 Now using 'CourseDataBase'.


In [11]:
def execute_sql(query: str, table_name: str, db_conn):
    """
    Executes an SQL query, handles errors, and displays the table structure beautifully for MySQL.

    Parameters:
        query (str): The SQL query to execute.
        table_name (str): The name of the table affected.
        db_conn (mysql.connector.MySQLConnection): Active MySQL database connection.
    """
    try:
        console.print(f"\n[cyan]⏳ Executing query for table: [bold]{table_name}[/bold]...[/cyan]", style="bold cyan")

        cursor = db_conn.cursor()
        cursor.execute(query)
        db_conn.commit()

        console.print(f"✅ [green]Success! Table '{table_name}' created or updated.[/green]", style="bold green")

        # Fetch table structure (using DESCRIBE for MySQL)
        cursor.execute(f"DESCRIBE {table_name};")
        columns = cursor.fetchall()

        if not columns:
            console.print(f"[red]❌ No table structure found for '{table_name}'![/red]", style="bold red")
            return

        # Display table structure using rich Table
        table = Table(show_header=True, header_style="bold yellow")
        table.add_column("Field", style="dim", width=20)
        table.add_column("Type", justify="left")
        table.add_column("Null", justify="center")
        table.add_column("Key", justify="center")
        table.add_column("Default", justify="left")
        table.add_column("Extra", justify="left")

        for col in columns:
            table.add_row(col[0], col[1], col[2], col[3], col[4] or "NULL", col[5] or "None")

        console.print("\n[bold yellow]📌 Table Structure:[/bold yellow]")
        console.print(table)

    except mysql.connector.Error as e:
        console.print(f"\n[red]❌ SQL Execution Failed![/red] Error: {e}", style="bold red")

    except Exception as e:
        console.print(f"\n[magenta]⚠️ Unexpected error: {e}[/magenta]", style="bold magenta")

    finally:
        cursor.close()
        console.print("[blue]🔍 Query execution complete.[/blue]", style="bold blue")

In [12]:
def delete_table(table_name: str, db_conn):
    """
    Deletes a table from the MySQL database with confirmation, error handling, and detailed output.

    Args:
        table_name (str): The name of the table to be deleted.
        db_conn (mysql.connector.MySQLConnection): The MySQL database connection.
    """
    try:
        cursor = db_conn.cursor()

        # Step 1: Confirm Deletion
        console.print(f"[bold yellow]⚠ WARNING:[/] You are about to delete the table [bold red]{table_name}[/].")
        confirmation = input("Are you sure? Type 'YES' to confirm: ")
        if confirmation.strip().upper() != "YES":
            console.print("[bold red]❌ Deletion canceled![/]")
            return

        start_time = time.time()

        # Step 2: Drop the table
        cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
        db_conn.commit()

        end_time = time.time()
        execution_time = end_time - start_time

        console.print(f"[bold green]✅ Successfully deleted the table '{table_name}' in {execution_time:.4f} seconds![/]")

        # Step 3: Display Remaining Tables
        cursor.execute("SHOW TABLES;")
        tables = cursor.fetchall()

        if tables:
            table = Table(title="📌 Remaining Tables in Database")
            table.add_column("Table Name", justify="left", style="cyan", no_wrap=True)
            for tbl in tables:
                table.add_row(tbl[0])
            console.print(table)
        else:
            console.print("[bold yellow]⚠ No tables left in the database![/]")

    except mysql.connector.Error as e:
        console.print(f"[bold red]❌ SQL Execution Failed![/] Error: {e}")

    finally:
        cursor.close()

In [13]:
delete_table("temp",db_conn)

⚠ WARNING: You are about to delete the table temp.

Are you sure? Type 'YES' to confirm: yes


✅ Successfully deleted the table 'temp' in 0.2618 seconds!

 📌 Remaining Tables in  
        Database         
┏━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Table Name            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━┩
│ assignments           │
│ certificates          │
│ chapter_content       │
│ chapters              │
│ content               │
│ course_chapters       │
│ course_reviews        │
│ courses               │
│ enrollments           │
│ interactions          │
│ password_resets       │
│ payments              │
│ qa                    │
│ questions             │
│ quizzes               │
│ student_progress      │
│ student_quiz_progress │
│ submissions           │
│ users                 │
└───────────────────────┘

## **Table 1 : Users Table `users`**  

| **Col_Name**       | **Type**                                | **Description**                                |
|-------------------|---------------------------------------------|-------------------------------------------------|
| `user_ID`      | INT | Unique User ID **`PRIMARY KEY`** |
| `full_name` | TEXT | User's full name |
| `email`| TEXT (Unique) | Email for Authenication|
| `pw_hash`| TEXT  | Hashed Password|
| `role`|TEXT  |`Student`, `Admin`, `Instructor` |
| `bio`|TEXT  | Short user bio|
| `linkedin_url`|TEXT (URL)  |Linkedin Profile |
| `github_url`|TEXT (URL)  |Github Profile|
| `profile_picture`|TEXT (URL)  |Optional Profile Picture |
| `pereferences`|JSON  |Stores Languaes, theme, notification settings |
| `education_level`|TEXT  |User's educational background (e.g., Bachelor's, Master's). Optional. |
|`years_of_experience`|INT|User's years of professional experience. Optional.|
|`upload_cv_url`|TEXT (URL)|URL where the user’s CV is uploaded. Optional.|
|`preferred_learning_platform`|TEXT|User's preferred platform for learning (e.g., Udemy, Coursera). Optional.|
|`location`|TEXT|User's current location (city, country). Optional.|
|`available_for_projects`|BOOLEAN|Whether the user is open to working on projects. Optional.|
|`programming_languages`|TEXT|Comma-separated list of programming languages the user is familiar with (e.g., Python, Java, C++, JavaScript).|
|`tools_and_technologies`|TEXT|Comma-separated list of tools and technologies the user is proficient in (e.g., Docker, AWS, Git, TensorFlow).|
|`interests`|TEXT|User’s professional interests or areas they wish to explore (e.g., Machine Learning, Web Development).|
|`certifications`|TEXT|Comma-separated list of relevant certifications the user holds (e.g., AWS Certified Solutions Architect, Google Data Engineer).|
|`preferred_project_types`|TEXT|Preferred types of projects or work the user is interested in (e.g., Backend Development, Data Science, Open Source).|
| `created_at`|TEXT(URL)  |Account Creation Date |
| `updated_at`|TIMESTAMPE  |Last Profile Updated Date |

---
* **Purpose**:
  - Stores user authentication details, roles, and preferences.
  - The skills and interests fields, along with other questions, help reduce the cold start problem by providing more information about the user.
  - By gathering details on programming languages, tools, interests, and certifications, the system can offer more personalized recommendations, improve user experience, and match users with suitable learning paths, projects, or jobs.
---

* **Query Code**
```
query = '''
CREATE TABLE IF NOT EXISTS users (
    user_ID UUID PRIMARY KEY DEFAULT gen_random_uuid(),  -- Auto-generated Unique User ID
    full_name TEXT NOT NULL,                             -- User's full name (required)
    email TEXT UNIQUE NOT NULL,                          -- User's email for authentication (unique)
    password_hash TEXT NOT NULL,                         -- Hashed password for security (required)
    role TEXT CHECK (role IN ('student', 'instructor', 'admin')) NOT NULL,  -- Role options (required)
    bio TEXT,                                           -- User's short biography (optional)
    linkedin_url TEXT,                                  -- LinkedIn profile URL (optional)
    github_url TEXT,                                    -- GitHub profile URL (optional)
    profile_picture TEXT,                               -- Profile picture URL (optional)
    preferences JSON,                                   -- User preferences (optional)
    education_level TEXT,                               -- User's educational background (optional)
    years_of_experience INT,                            -- Years of professional experience (optional)
    upload_cv_url TEXT,                                 -- URL to the user's uploaded CV (optional)
    preferred_learning_platform TEXT,                   -- Preferred platform for learning (optional)
    location TEXT,                                      -- User's current location (optional)
    available_for_projects BOOLEAN,                      -- Availability for projects (optional)
    programming_languages TEXT,                         -- List of programming languages the user knows (optional)
    tools_and_technologies TEXT,                        -- Tools and technologies the user is proficient in (optional)
    interests TEXT,                                     -- User’s professional interests (optional)
    preferred_project_types TEXT,                       -- Types of projects the user prefers (optional)
    created_at TIMESTAMP DEFAULT NOW(),                 -- Account creation timestamp
    updated_at TIMESTAMP DEFAULT NOW()                  -- Last profile update timestamp
);
'''
```





In [23]:
user_query = '''
CREATE TABLE IF NOT EXISTS users (
    user_ID CHAR(36) PRIMARY KEY DEFAULT (UUID()),  -- Use CHAR(36) and UUID() function for UUID generation
    full_name TEXT NOT NULL,                         -- User's full name (required)
    email VARCHAR(255) UNIQUE NOT NULL,              -- User's email for authentication (unique)
    password_hash TEXT NOT NULL,                     -- Hashed password for security (required)
    role TEXT CHECK (role IN ('student', 'instructor', 'admin')) NOT NULL,  -- Role options (required)
    bio TEXT,                                       -- User's short biography (optional)
    linkedin_url TEXT,                              -- LinkedIn profile URL (optional)
    github_url TEXT,                                -- GitHub profile URL (optional)
    profile_picture TEXT,                           -- Profile picture URL (optional)
    preferences JSON,                               -- User preferences (optional)
    education_level TEXT,                           -- User's educational background (optional)
    years_of_experience INT,                        -- Years of professional experience (optional)
    upload_cv_url TEXT,                             -- URL to the user's uploaded CV (optional)
    preferred_learning_platform TEXT,               -- Preferred platform for learning (optional)
    location TEXT,                                  -- User's current location (optional)
    available_for_projects BOOLEAN,                  -- Availability for projects (optional)
    programming_languages TEXT,                     -- List of programming languages the user knows (optional)
    tools_and_technologies TEXT,                    -- Tools and technologies the user is proficient in (optional)
    interests TEXT,                                 -- User’s professional interests (optional)
    preferred_project_types TEXT,                   -- Types of projects the user prefers (optional)
    created_at TIMESTAMP DEFAULT NOW(),             -- Account creation timestamp
    updated_at TIMESTAMP DEFAULT NOW()              -- Last profile update timestamp
);
'''

execute_sql(user_query, "users", db_conn)

⏳ Executing query for table: users...

✅ Success! Table 'users' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type         ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_ID              │ char(36)     │  NO  │ PRI │ uuid()            │ DEFAULT_GENERATED │
│ full_name            │ text         │  NO  │     │ NULL              │ None              │
│ email                │ varchar(255) │  NO  │ UNI │ NULL              │ None              │
│ password_hash        │ text         │  NO  │     │ NULL              │ None              │
│ role                 │ text         │  NO  │     │ NULL              │ None              │
│ bio                  │ text         │ YES  │     │ NULL              │ None              │
│ linkedin_url         │ text         │ YES  │     │ NULL              │ None              │
│ github_url           │ text         │ YES  │     │ NULL              │ None              │
│ profile_picture      │ text         │ YES  │     │ NULL              │ None              │
│ preferences          │ json         │ YES  │     │ NULL              │ None              │
│ education_level      │ text         │ YES  │     │ NULL              │ None              │
│ years_of_experience  │ int          │ YES  │     │ NULL              │ None              │
│ upload_cv_url        │ text         │ YES  │     │ NULL              │ None              │
│ preferred_learning_… │ text         │ YES  │     │ NULL              │ None              │
│ location             │ text         │ YES  │     │ NULL              │ None              │
│ available_for_proje… │ tinyint(1)   │ YES  │     │ NULL              │ None              │
│ programming_languag… │ text         │ YES  │     │ NULL              │ None              │
│ tools_and_technolog… │ text         │ YES  │     │ NULL              │ None              │
│ interests            │ text         │ YES  │     │ NULL              │ None              │
│ preferred_project_t… │ text         │ YES  │     │ NULL              │ None              │
│ created_at           │ timestamp    │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
│ updated_at           │ timestamp    │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
│ email_verification_… │ varchar(255) │ YES  │     │ NULL              │ None              │
│ verified             │ tinyint(1)   │ YES  │     │ 0                 │ None              │
│ email_verification_… │ datetime     │ YES  │     │ NULL              │ None              │
└──────────────────────┴──────────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 2 : Courses  Table `courses`**  

| **Col_Name**       | **Type**                                | **Description**                                |
|-------------------|---------------------------------------------|-------------------------------------------------|
| `couse_ID`      | VARCHAR(16) | Unique Course ID **`PRIMARY KEY`** |
| `course_title` | TEXT | Course Title |
| `description`| TEXT | Detailed Course Info|
| `level`| TEXT  | `Beginner`, `Intermediate` or `Advanced`|
| `category`|TEXT  |eg. `AI`, `Data Science`, `Cybersecuirty` ... |
| `instructor_ID`|INT (FK)  | Links to `users.user_ID`|
| `price`|FLOAT  |Course Price (if paid)|
| `thumnail_url`|TEXT (URL)  |Course Image URL|
| `total_hours`|FLOAT  |Course length in hours |
| `created_at`|TIMESTAMPE  |Account Creation Date |
| `updated_at`|TIMESTAMP |Last Updated Date |
| `what_you_will_learn` | TEXT| What will you learn in this course|
| `who_this_course_is_for` | TEXT| Who should enroll this course|


---
* **Purpose**:
  - Stores all available courses, topics, and difficulty levels.
---

* **Query Code**
```
query = '''
CREATE TABLE IF NOT EXISTS courses (
    course_ID VARCHAR(16) PRIMARY KEY,
    course_title TEXT NOT NULL,
    description TEXT NOT NULL,
    category TEXT NOT NULL,
    level TEXT CHECK (level IN ('Beginner', 'Intermediate', 'Advanced')) NOT NULL,
    instructor_id TEXT REFERENCES users(user_ID) ON DELETE CASCADE,
    price REAL DEFAULT 0.0,
    thumbnail_url TEXT,
    total_hours REAL DEFAULT 0,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    what_you_will_learn TEXT DEFAULT '',  -- Added 'What you'll learn' field
    who_this_course_is_for TEXT DEFAULT ''  -- Added 'Who this course is for' field
);
'''
```




In [ ]:
courses_query = '''
CREATE TABLE courses (
    course_ID CHAR(36) PRIMARY KEY, -- Ensuring UUID consistency
    course_title TEXT NOT NULL,
    description TEXT NOT NULL,
    category TEXT NOT NULL,
    level TEXT CHECK (level IN ('Beginner', 'Intermediate', 'Advanced')) NOT NULL,
    instructor_id CHAR(36) NOT NULL, -- Instructor ID without direct REFERENCES
    price REAL DEFAULT 0.0,
    thumbnail_url TEXT,
    total_hours REAL DEFAULT 0,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    what_you_will_learn TEXT DEFAULT '',  -- Added 'What you'll learn' field
    who_this_course_is_for TEXT DEFAULT '',  -- Added 'Who this course is for' field
    FOREIGN KEY (instructor_id) REFERENCES users(user_ID) ON DELETE CASCADE
);
'''

execute_sql(courses_query, "courses", db_conn)

⏳ Executing query for table: courses...

✅ Success! Table 'courses' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type     ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ course_ID            │ char(36) │  NO  │ PRI │ NULL              │ None              │
│ course_title         │ text     │  NO  │     │ NULL              │ None              │
│ description          │ text     │  NO  │     │ NULL              │ None              │
│ category             │ text     │  NO  │     │ NULL              │ None              │
│ level                │ text     │  NO  │     │ NULL              │ None              │
│ instructor_id        │ char(36) │  NO  │ MUL │ NULL              │ None              │
│ price                │ double   │ YES  │     │ 0                 │ None              │
│ thumbnail_url        │ text     │ YES  │     │ NULL              │ None              │
│ total_hours          │ double   │ YES  │     │ 0                 │ None              │
│ created_at           │ datetime │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
│ updated_at           │ datetime │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
│ what_you_will_learn  │ text     │ YES  │     │ NULL              │ None              │
│ who_this_course_is_… │ text     │ YES  │     │ NULL              │ None              │
└──────────────────────┴──────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 3 : Chapter  Table `chapters`**  

| **Col_Name**       | **Type**                                | **Description**                                |
|-------------------|---------------------------------------------|-------------------------------------------------|
| `chapter_ID  `      | INT | Unique Chapter ID **`PRIMARY KEY`** |
| `title  ` | VARCHAR(255) | Chapter Title |
| `description  `| TEXT | Detailed Chapter Info|
| `created_at `| DATETIME   | CURRENT_TIMESTAMP|

| **Col_Name**       | **Type**                                | **Description**                                |
|-------------------|---------------------------------------------|-------------------------------------------------|
| `course_ID   `      | INT | Links to `courses.CourseID` |
| `chapter_ID   ` | INT | Links to `chapters.chapterID` |
| `position   `| INT | The order of the chapter in the course|


---
* **Purpose**:
  - Stores chapters that belong to one or more courses.
  - Each chapter can appear in multiple courses with different positions (ordering).
---




In [ ]:
chapters_query = '''
CREATE TABLE IF NOT EXISTS chapters (
    chapter_ID INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    description TEXT NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
'''

execute_sql(chapters_query, "chapters", db_conn)

⏳ Executing query for table: chapters...

✅ Success! Table 'chapters' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type         ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ chapter_ID           │ int          │  NO  │ PRI │ NULL              │ auto_increment    │
│ title                │ varchar(255) │  NO  │     │ NULL              │ None              │
│ description          │ text         │  NO  │     │ NULL              │ None              │
│ created_at           │ datetime     │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
└──────────────────────┴──────────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

In [ ]:
course_chapters_query = '''
CREATE TABLE IF NOT EXISTS course_chapters (
    course_ID CHAR(36),
    chapter_ID INT,
    position INT,
    PRIMARY KEY (course_ID, chapter_ID),
    FOREIGN KEY (course_ID) REFERENCES courses(course_ID) ON DELETE CASCADE,
    FOREIGN KEY (chapter_ID) REFERENCES chapters(chapter_ID) ON DELETE CASCADE
);
'''

execute_sql(course_chapters_query, "course_chapters", db_conn)

⏳ Executing query for table: course_chapters...

✅ Success! Table 'course_chapters' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━┓
┃ Field                ┃ Type     ┃ Null ┃ Key ┃ Default ┃ Extra ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━┩
│ course_ID            │ char(36) │  NO  │ PRI │ NULL    │ None  │
│ chapter_ID           │ int      │  NO  │ PRI │ NULL    │ None  │
│ position             │ int      │ YES  │     │ NULL    │ None  │
└──────────────────────┴──────────┴──────┴─────┴─────────┴───────┘

🔍 Query execution complete.

## **Table 4 : content   Table `content `**  


# **course_content  Table**

| **Col_Name**       | **Type**                                | **Description**                                |
|-------------------|---------------------------------------------|-------------------------------------------------|
| `content_ID   `      | INT | Unique Content ID **`PRIMARY KEY`** |
| `content_type  ` | ENUM | ENUM(`video`, `pdf`, `quiz`, `ppt`, `text`, `file`) |
| `content_url   `| VARCHAR(255) | The Link of the content|
| `description  `| TEXT| The Description of the content|
| `duration   `| REAL   | The Duration in case it was a video|
| `created_at    `| DATETIME   | CURRENT_TIMESTAMP|

---

# **Chapter Content**

| **Col_Name**       | **Type**                                | **Description**                                |
|-------------------|---------------------------------------------|-------------------------------------------------|
| `chapter_ID   `      | INT | Links to `chapters.CourseID` |
| `content_ID    ` | INT | Links to `contents.chapterID` |
| `position    `| INT | The order of the chapter in the course|


---
* **Purpose**:
  - Stores content (videos, PDFs, quizzes, etc.) that belongs to one or more chapters.
  - Content must have a position within each chapter.
---


```
CREATE TABLE IF NOT EXISTS content (
    content_ID INT AUTO_INCREMENT PRIMARY KEY,
    content_type ENUM('video', 'pdf', 'quiz', 'ppt', 'text', 'file') NOT NULL,
    content_url VARCHAR(255) NOT NULL UNIQUE,  
    description TEXT,  -- ✅ Added description for the content
    duration REAL DEFAULT 0,  
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS chapter_content (
    chapter_ID INT NOT NULL,
    content_ID INT NOT NULL,
    position INT NOT NULL,  -- ✅ Position of the content in each chapter
    PRIMARY KEY (chapter_ID, content_ID),
    FOREIGN KEY (chapter_ID) REFERENCES chapters(chapter_ID) ON DELETE CASCADE,
    FOREIGN KEY (content_ID) REFERENCES content(content_ID) ON DELETE CASCADE
);

```

In [ ]:
contents_query = '''
CREATE TABLE IF NOT EXISTS content (
    content_ID INT AUTO_INCREMENT PRIMARY KEY,
    content_type ENUM('video', 'pdf', 'quiz', 'ppt', 'text', 'file') NOT NULL,
    content_url VARCHAR(255) NOT NULL UNIQUE,
    description TEXT,
    duration REAL DEFAULT 0,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);
'''

execute_sql(contents_query, "content", db_conn)

⏳ Executing query for table: content...

✅ Success! Table 'content' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type                                ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ content_ID           │ int                                 │  NO  │ PRI │ NULL              │ auto_increment    │
│ content_type         │ enum('video','pdf','quiz','ppt','t… │  NO  │     │ NULL              │ None              │
│ content_url          │ varchar(255)                        │  NO  │ UNI │ NULL              │ None              │
│ description          │ text                                │ YES  │     │ NULL              │ None              │
│ duration             │ double                              │ YES  │     │ 0                 │ None              │
│ created_at           │ datetime                            │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
└──────────────────────┴─────────────────────────────────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

In [ ]:
chapter_content_query = '''
CREATE TABLE IF NOT EXISTS chapter_content (
    chapter_ID INT NOT NULL,
    content_ID INT NOT NULL,
    position INT NOT NULL,
    PRIMARY KEY (chapter_ID, content_ID),
    FOREIGN KEY (chapter_ID) REFERENCES chapters(chapter_ID) ON DELETE CASCADE,
    FOREIGN KEY (content_ID) REFERENCES content(content_ID) ON DELETE CASCADE
);
'''

execute_sql(chapter_content_query, "chapter_content", db_conn)

⏳ Executing query for table: chapter_content...

✅ Success! Table 'chapter_content' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━┓
┃ Field                ┃ Type ┃ Null ┃ Key ┃ Default ┃ Extra ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━┩
│ chapter_ID           │ int  │  NO  │ PRI │ NULL    │ None  │
│ content_ID           │ int  │  NO  │ PRI │ NULL    │ None  │
│ position             │ int  │  NO  │     │ NULL    │ None  │
└──────────────────────┴──────┴──────┴─────┴─────────┴───────┘

🔍 Query execution complete.

## **Table 5 : Student Progress**  

| **Col_Name**       | **Type**                                | **Description**                                |
|-------------------|---------------------------------------------|-------------------------------------------------|
| `progress_ID  `      | INT | Unique Progress ID **`PRIMARY KEY`** |
| `user_ID  ` | CHAR(36) | Links to `users.UserID` |
| `course_ID  `| VARCHAR(16) | Links to `courses.CourseID`|
| `chapter_ID `| INT   | Links to `chapters.chapterID`|
| `content_ID `      | INT | Links to `content.contentID` |
| `completed  ` | BOOLEAN | weather the lesson is completed or not |
| `last_updated`| DATETIME| Last Updated Date|


---
* **Purpose**:
  - Tracks student progress per lesson (content).
  - Green check ✅ appears when a student completes a lesson.
  - Progress updates automatically in all courses containing that lesson
---


```
CREATE TABLE IF NOT EXISTS student_progress (
    progress_ID INT AUTO_INCREMENT PRIMARY KEY,
    user_ID CHAR(36) NOT NULL,
    course_ID VARCHAR(16) NOT NULL,
    chapter_ID INT NOT NULL,
    content_ID INT NOT NULL,  
    completed BOOLEAN DEFAULT FALSE,
    last_updated DATETIME DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    FOREIGN KEY (user_ID) REFERENCES users(user_ID) ON DELETE CASCADE,
    FOREIGN KEY (course_ID) REFERENCES courses(course_ID) ON DELETE CASCADE,
    FOREIGN KEY (chapter_ID) REFERENCES chapters(chapter_ID) ON DELETE CASCADE,
    FOREIGN KEY (content_ID) REFERENCES content(content_ID) ON DELETE CASCADE
);



```

In [ ]:
student_progress_query = '''
CREATE TABLE IF NOT EXISTS student_progress (
    progress_ID INT AUTO_INCREMENT PRIMARY KEY,
    user_ID CHAR(36) NOT NULL,
    course_ID CHAR(36) NOT NULL,
    chapter_ID INT NOT NULL,
    content_ID INT NOT NULL,
    completed BOOLEAN DEFAULT FALSE,
    last_updated DATETIME DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    FOREIGN KEY (user_ID) REFERENCES users(user_ID) ON DELETE CASCADE,
    FOREIGN KEY (course_ID) REFERENCES courses(course_ID) ON DELETE CASCADE,
    FOREIGN KEY (chapter_ID) REFERENCES chapters(chapter_ID) ON DELETE CASCADE,
    FOREIGN KEY (content_ID) REFERENCES content(content_ID) ON DELETE CASCADE
);
'''

execute_sql(student_progress_query, "student_progress", db_conn)

⏳ Executing query for table: student_progress...

✅ Success! Table 'student_progress' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type       ┃ Null ┃ Key ┃ Default           ┃ Extra                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ progress_ID          │ int        │  NO  │ PRI │ NULL              │ auto_increment                             │
│ user_ID              │ char(36)   │  NO  │ MUL │ NULL              │ None                                       │
│ course_ID            │ char(36)   │  NO  │ MUL │ NULL              │ None                                       │
│ chapter_ID           │ int        │  NO  │ MUL │ NULL              │ None                                       │
│ content_ID           │ int        │  NO  │ MUL │ NULL              │ None                                       │
│ completed            │ tinyint(1) │ YES  │     │ 0                 │ None                                       │
│ last_updated         │ datetime   │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED on update                │
│                      │            │      │     │                   │ CURRENT_TIMESTAMP                          │
└──────────────────────┴────────────┴──────┴─────┴───────────────────┴────────────────────────────────────────────┘

🔍 Query execution complete.

## **Table 6 : Enrollments  Table `enrollments`**  

| **Col_Name**       | **Type**                                | **Description**                                |
|-------------------|---------------------------------------------|-------------------------------------------------|
| `enroll_ID`      | INT | Unique enrollment ID **`PRIMARY KEY`** |
| `user_ID` | CHAR(36) (FK) | links to `users.user_ID` |
| `course_ID`| INT (FK) | links to `courses.course_ID`|
| `status`| TEXT  | `Unpaid`, `Active`, `Completed`|
| `progress`|FLOAT  |Percentage Progress (0~100%) |
| `started_at`|TIMESTAMP  | Enrollment Start Date|

---
* **Purpose**:
  - Tracks which users are enrolled in which courses.
---

* **Query Code**
```
query = '''
CCREATE TABLE IF NOT EXISTS enrollments (
    enroll_ID CHAR(36) PRIMARY KEY DEFAULT (UUID()),                        -- Auto-generated Unique Enrollment ID
    user_ID CHAR(36) NOT NULL,                                              -- User ID (Foreign Key)
    course_ID VARCHAR(16) NOT NULL,                                         -- Course ID (Foreign Key)
    status ENUM('Unenrolled', 'Active', 'Completed') DEFAULT 'Unenrolled',  -- Status of the enrollment
    progress FLOAT DEFAULT 0.0,                                             -- Progress of the user in the course
    started_at DATETIME DEFAULT CURRENT_TIMESTAMP,                          -- Timestamp when the user started the course
    FOREIGN KEY (user_ID) REFERENCES users(user_ID) ON DELETE CASCADE,       -- Foreign Key to users table
    FOREIGN KEY (course_ID) REFERENCES courses(course_ID) ON DELETE CASCADE  -- Foreign Key to courses table
);
'''
```




In [ ]:
enroll_query = '''
CREATE TABLE IF NOT EXISTS enrollments (
    enroll_ID CHAR(36) PRIMARY KEY DEFAULT (UUID()),                        -- Auto-generated Unique Enrollment ID
    user_ID CHAR(36) NOT NULL,                                              -- User ID (Foreign Key)
    course_ID CHAR(36) NOT NULL,                                            -- Course ID (Foreign Key)
    status ENUM('Unenrolled', 'Active', 'Completed') DEFAULT 'Unenrolled',  -- Status of the enrollment
    progress FLOAT DEFAULT 0.0,                                             -- Progress of the user in the course
    started_at DATETIME DEFAULT CURRENT_TIMESTAMP,                          -- Timestamp when the user started the course
    FOREIGN KEY (user_ID) REFERENCES users(user_ID) ON DELETE CASCADE,       -- Foreign Key to users table
    FOREIGN KEY (course_ID) REFERENCES courses(course_ID) ON DELETE CASCADE  -- Foreign Key to courses table
);


'''
execute_sql(enroll_query, "enrollments", db_conn)

⏳ Executing query for table: enrollments...

✅ Success! Table 'enrollments' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type                                ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ enroll_ID            │ char(36)                            │  NO  │ PRI │ uuid()            │ DEFAULT_GENERATED │
│ user_ID              │ char(36)                            │  NO  │ MUL │ NULL              │ None              │
│ course_ID            │ char(36)                            │  NO  │ MUL │ NULL              │ None              │
│ status               │ enum('Unenrolled','Active','Comple… │ YES  │     │ Unenrolled        │ None              │
│ progress             │ float                               │ YES  │     │ 0                 │ None              │
│ started_at           │ datetime                            │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
└──────────────────────┴─────────────────────────────────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 5 : Quizzes  Table `quizzes`**  

| **Col_Name**|**Type** | **Description**|
|-------------|--------------------------|-----------------------------|
| `quiz_ID `      | INT (PK) | Unique Interaction ID **`PRIMARY KEY`** |
| `content_ID` | INT (FK) | links to `content.content_ID` |
| `title`| TEXT | Quiz Title|
| `description `| TIMESTAMP  | Creation Date|
|`pass_perecentage`|FLOAT|The Perecentage to pass the quiz|
|`total_score`|INT|The total score of the quiz|
| `created_at `|TIMEASTAMP  |Updated Date |

---
* **Purpose**:
  - This table will store details about the quiz itself. Each quiz will be linked to a chapter through the chapter_contents table, and we can add a position field to specify where the quiz appears in the chapter.
  - The table contains the title and description of the quiz as well as the total score and passing critera of the quiz
---




In [ ]:
quiz_query = '''
CREATE TABLE IF NOT EXISTS quizzes (
    quiz_ID INT AUTO_INCREMENT PRIMARY KEY,
    content_ID INT NOT NULL,
    title TEXT NOT NULL,
    description TEXT NOT NULL,
    pass_percentage FLOAT NOT NULL,
    total_score INT NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (content_ID) REFERENCES content(content_ID) ON DELETE CASCADE
);
'''
execute_sql(quiz_query, "quizzes", db_conn)

⏳ Executing query for table: quizzes...

✅ Success! Table 'quizzes' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type     ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ quiz_ID              │ int      │  NO  │ PRI │ NULL              │ auto_increment    │
│ content_ID           │ int      │  NO  │ MUL │ NULL              │ None              │
│ title                │ text     │  NO  │     │ NULL              │ None              │
│ description          │ text     │  NO  │     │ NULL              │ None              │
│ pass_percentage      │ float    │  NO  │     │ NULL              │ None              │
│ total_score          │ int      │  NO  │     │ NULL              │ None              │
│ created_at           │ datetime │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
└──────────────────────┴──────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 6 : Questions  Table `questions`**  

| **Col_Name**|**Type** | **Description**|
|-------------|--------------------------|-----------------------------|
| `question_ID`      | INT (PK) | Unique question ID **`PRIMARY KEY`** |
| `quiz_ID` | INT (FK) | links to `quizes.quiz_ID` |
| `question_text`| TEXT | Question Content|
| `type`| TEXT  | `MCQ`, `Short Answer`, `True/False`|
| `correct_answer`|TEXT  |Correct Answer |
|`created_at`|TIMESTAMP|creation date|
|`updated_at`|TIMESTAMP|updated date|

---
* **Purpose**:
  - This table will store the questions for each quiz, including the question type (MCQ, True/False).
---

* **Query Code**
```
query = '''
CREATE TABLE questions (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    quiz_id UUID REFERENCES quizzes(id) ON DELETE CASCADE,
    question_text TEXT NOT NULL,
    type TEXT CHECK (type IN ('MCQ', 'Short Answer', 'True/False')) NOT NULL,
    correct_answer TEXT NOT NULL,
    created_at TIMESTAMP DEFAULT NOW(),
    updated_at TIMESTAMP DEFAULT NOW()
);
'''
```




In [ ]:
questions_query = '''
CREATE TABLE IF NOT EXISTS questions (
    question_ID CHAR(36) PRIMARY KEY DEFAULT (UUID()),  -- Use UUID for unique question IDs
    quiz_ID INT NOT NULL,
    question_text TEXT NOT NULL,
    type TEXT CHECK (type IN ('MCQ', 'Short Answer', 'True/False')) NOT NULL,
    correct_answer TEXT NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (quiz_ID) REFERENCES quizzes(quiz_ID) ON DELETE CASCADE
);
'''
execute_sql(questions_query, "questions", db_conn)

⏳ Executing query for table: questions...

✅ Success! Table 'questions' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type     ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ question_ID          │ char(36) │  NO  │ PRI │ uuid()            │ DEFAULT_GENERATED │
│ quiz_ID              │ int      │  NO  │ MUL │ NULL              │ None              │
│ question_text        │ text     │  NO  │     │ NULL              │ None              │
│ type                 │ text     │  NO  │     │ NULL              │ None              │
│ correct_answer       │ text     │  NO  │     │ NULL              │ None              │
│ created_at           │ datetime │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
│ updated_at           │ datetime │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
└──────────────────────┴──────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 7 : student_quiz_progress**  

| **Col_Name**|**Type** | **Description**|
|-------------|--------------------------|-----------------------------|
| `progress_ID `      | INT (PK) | Unique Progress ID **`PRIMARY KEY`** |
| `user_ID ` | CHAR(36) (FK) | links to `users.cuser_ID` |
| `quiz_ID `| INT | links to `quizzes.quiz_ID`|
| `attempt_number `| INT  | The attempt number of the student|
| `score `|FLOAT  |the score of the student |
|`passed `|BOOLEAN|The status of the quiz `passed` or `failed`|
|`completed_at `|FLOAT|The grade to pass the Assignment|

---
* **Purpose**:
  - This table will track each student's quiz attempts, score, and pass/fail status. The student can attempt the quiz up to 3 times.
---

* **Query Code**
```
query = '''
CREATE TABLE IF NOT EXISTS student_quiz_progress (
    progress_ID INT AUTO_INCREMENT PRIMARY KEY,
    user_ID CHAR(36) NOT NULL,
    quiz_ID INT NOT NULL,
    attempt_number INT DEFAULT 1,
    score FLOAT DEFAULT 0.0,
    passed BOOLEAN DEFAULT FALSE,
    completed_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_ID) REFERENCES users(user_ID) ON DELETE CASCADE,
    FOREIGN KEY (quiz_ID) REFERENCES quizzes(quiz_ID) ON DELETE CASCADE
);





In [ ]:
student_progress_query = '''
CREATE TABLE IF NOT EXISTS student_quiz_progress (
    progress_ID INT AUTO_INCREMENT PRIMARY KEY,
    user_ID CHAR(36) NOT NULL,
    quiz_ID INT NOT NULL,
    attempt_number INT DEFAULT 1,
    score FLOAT DEFAULT 0.0,
    passed BOOLEAN DEFAULT FALSE,
    completed_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_ID) REFERENCES users(user_ID) ON DELETE CASCADE,
    FOREIGN KEY (quiz_ID) REFERENCES quizzes(quiz_ID) ON DELETE CASCADE
);
'''
execute_sql(student_progress_query, "student_quiz_progress", db_conn)

⏳ Executing query for table: student_quiz_progress...

✅ Success! Table 'student_quiz_progress' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type       ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ progress_ID          │ int        │  NO  │ PRI │ NULL              │ auto_increment    │
│ user_ID              │ char(36)   │  NO  │ MUL │ NULL              │ None              │
│ quiz_ID              │ int        │  NO  │ MUL │ NULL              │ None              │
│ attempt_number       │ int        │ YES  │     │ 1                 │ None              │
│ score                │ float      │ YES  │     │ 0                 │ None              │
│ passed               │ tinyint(1) │ YES  │     │ 0                 │ None              │
│ completed_at         │ datetime   │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
└──────────────────────┴────────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 8 : Assignment  Table `assignments`**  

| **Col_Name**|**Type** | **Description**|
|-------------|--------------------------|-----------------------------|
| `assignment_ID`      | INT (PK) | Unique Assignment ID **`PRIMARY KEY`** |
| `course_ID` | INT (FK) | links to `courses.course_ID` |
| `title`| TEXT | Assignment Title|
| `description`| TEXT  | Instructions|
| `deadline`|TIMEATAMP  |Submission Deadline |
|`created_at`|TIMESTAMP|Creation Date|
|`scope_pass_grade`|FLOAT|The grade to pass the Assignment|

---
* **Purpose**:
  - Stores Assignments of the course
---

* **Query Code**
```
query = '''
CREATE TABLE IF NOT EXISTS assignments (
    assignment_ID CHAR(36) PRIMARY KEY DEFAULT (UUID()),  -- Use UUID for unique assignment IDs
    course_ID CHAR(36) NOT NULL,  -- Use CHAR(36) for course_ID (UUID format)
    title TEXT NOT NULL,
    description TEXT NOT NULL,
    deadline DATETIME,
    pass_grade INT NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (course_ID) REFERENCES courses(course_ID) ON DELETE CASCADE
);





In [ ]:
assignment_query = '''
CREATE TABLE IF NOT EXISTS assignments (
    assignment_ID CHAR(36) PRIMARY KEY DEFAULT (UUID()),  -- Use UUID for unique assignment IDs
    course_ID CHAR(36) NOT NULL,  -- Use CHAR(36) for course_ID (UUID format)
    title TEXT NOT NULL,
    description TEXT NOT NULL,
    deadline DATETIME,
    pass_grade INT NOT NULL,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (course_ID) REFERENCES courses(course_ID) ON DELETE CASCADE
);

'''
execute_sql(assignment_query, "assignments", db_conn)

⏳ Executing query for table: assignments...

✅ Success! Table 'assignments' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type     ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ assignment_ID        │ char(36) │  NO  │ PRI │ uuid()            │ DEFAULT_GENERATED │
│ course_ID            │ char(36) │  NO  │ MUL │ NULL              │ None              │
│ title                │ text     │  NO  │     │ NULL              │ None              │
│ description          │ text     │  NO  │     │ NULL              │ None              │
│ deadline             │ datetime │ YES  │     │ NULL              │ None              │
│ pass_grade           │ int      │  NO  │     │ NULL              │ None              │
│ created_at           │ datetime │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
└──────────────────────┴──────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 9 : Submissions  Table `submissions`**  

| **Col_Name**|**Type** | **Description**|
|-------------|--------------------------|-----------------------------|
| `submission_ID`      | INT (PK) | Unique Submission ID **`PRIMARY KEY`** |
| `user_ID` | INT (FK) | links to `users.user_ID` |
| `assignment_ID`| INT (FK)  | links to `assignments.assignment_ID`|
| `file_URL`| TEXT  | Link to Submission|
| `grade`|FLOAT  |Score given by the instructor |
|`submitted_at`|TIMESTAMP|Submission Time|
|`attempts`|INT|Attempts (0,1,2)|
|`status`|TEXT|`Not Submitted`, `Pending`, `Failed`, `Passed`|

---
* **Purpose**:
  - Tracks student assignment submissions.
---

* **Query Code**
```
query = '''
CREATE TABLE IF NOT EXISTS submissions (
    submission_ID CHAR(36) PRIMARY KEY DEFAULT (UUID()),  -- Using UUID() for unique submission IDs
    user_ID CHAR(36) NOT NULL,  -- Use CHAR(36) for user_ID (UUID format)
    assignment_ID CHAR(36) NOT NULL,  -- Use CHAR(36) for assignment_ID (UUID format)
    file_url TEXT NOT NULL,
    grade FLOAT DEFAULT NULL,
    submitted_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    attempts INT DEFAULT 0 CHECK (attempts >= 0),  -- Ensure attempts are non-negative
    status TEXT CHECK (status IN ('Not Submitted', 'Pending', 'Failed', 'Passed')),  -- No default value for status
    FOREIGN KEY (user_ID) REFERENCES users(user_ID) ON DELETE CASCADE,
    FOREIGN KEY (assignment_ID) REFERENCES assignments(assignment_ID) ON DELETE CASCADE
);
'''
```




In [ ]:
submissions_query = '''
CREATE TABLE IF NOT EXISTS submissions (
    submission_ID CHAR(36) PRIMARY KEY DEFAULT (UUID()),
    user_ID CHAR(36) NOT NULL,
    assignment_ID CHAR(36) NOT NULL,
    file_url TEXT NOT NULL,
    grade FLOAT DEFAULT NULL,
    submitted_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    attempts INT DEFAULT 0 CHECK (attempts >= 0),  -- Ensure attempts are non-negative
    status TEXT CHECK (status IN ('Not Submitted', 'Pending', 'Failed', 'Passed')),  -- No default value for status
    FOREIGN KEY (user_ID) REFERENCES users(user_ID) ON DELETE CASCADE,
    FOREIGN KEY (assignment_ID) REFERENCES assignments(assignment_ID) ON DELETE CASCADE
);
'''
execute_sql(submissions_query, "submissions", db_conn)

⏳ Executing query for table: submissions...

✅ Success! Table 'submissions' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type     ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ submission_ID        │ char(36) │  NO  │ PRI │ uuid()            │ DEFAULT_GENERATED │
│ user_ID              │ char(36) │  NO  │ MUL │ NULL              │ None              │
│ assignment_ID        │ char(36) │  NO  │ MUL │ NULL              │ None              │
│ file_url             │ text     │  NO  │     │ NULL              │ None              │
│ grade                │ float    │ YES  │     │ NULL              │ None              │
│ submitted_at         │ datetime │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
│ attempts             │ int      │ YES  │     │ 0                 │ None              │
│ status               │ text     │ YES  │     │ NULL              │ None              │
└──────────────────────┴──────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 10 : QA  Table `qa`**  

| **Col_Name**|**Type** | **Description**|
|-------------|--------------------------|-----------------------------|
| `question_ID`      | INT (PK) | Unique QA ID **`PRIMARY KEY`** |
| `course_ID` | INT (FK) | links to `courses.course_ID` |
| `user_ID`| INT (FK)  | links to `users.user_ID`|
| `instructor_ID`| INT (FK)  | links to `users.user_ID`|
| `question`|TEXT  |Question Asked |
|`answer`|TEXT|Instructor's Answer|
|`created_at`|TIMESTAMP|Creation Date|

---
* **Purpose**:
  - Stores student-instructor discussions for each course.
---

* **Query Code**
```
query = '''
CREATE TABLE qa (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    course_id UUID REFERENCES courses(id) ON DELETE CASCADE,
    student_id UUID REFERENCES users(id) ON DELETE CASCADE,
    instructor_id UUID REFERENCES users(id) ON DELETE CASCADE,
    question TEXT NOT NULL,
    answer TEXT,
    created_at TIMESTAMP DEFAULT NOW()
);
'''
```




In [ ]:
qa_query = '''
CREATE TABLE IF NOT EXISTS qa (
    question_ID CHAR(36) PRIMARY KEY, -- UUID stored as CHAR(36)
    course_ID CHAR(36) NOT NULL, -- Ensuring foreign key consistency
    student_ID CHAR(36) NOT NULL, -- Ensuring student reference consistency
    instructor_ID CHAR(36) DEFAULT NULL, -- Instructor ID may be NULL initially
    question TEXT NOT NULL,
    answer TEXT DEFAULT NULL, -- Answer can be NULL initially
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP, -- Uses MySQL's timestamp handling
    FOREIGN KEY (course_ID) REFERENCES courses(course_ID) ON DELETE CASCADE, -- Enforcing course linkage
    FOREIGN KEY (student_ID) REFERENCES users(user_ID) ON DELETE CASCADE, -- Enforcing student association
    FOREIGN KEY (instructor_ID) REFERENCES users(user_ID) ON DELETE CASCADE -- Optional instructor association
);
'''
execute_sql(qa_query, "qa", db_conn)

⏳ Executing query for table: qa...

✅ Success! Table 'qa' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type     ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ question_ID          │ char(36) │  NO  │ PRI │ NULL              │ None              │
│ course_ID            │ char(36) │  NO  │ MUL │ NULL              │ None              │
│ student_ID           │ char(36) │  NO  │ MUL │ NULL              │ None              │
│ instructor_ID        │ char(36) │ YES  │ MUL │ NULL              │ None              │
│ question             │ text     │  NO  │     │ NULL              │ None              │
│ answer               │ text     │ YES  │     │ NULL              │ None              │
│ created_at           │ datetime │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
└──────────────────────┴──────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 11 : Chatbot Interactions `interactions`**  

| **Col_Name**|**Type** | **Description**|
|-------------|--------------------------|-----------------------------|
| `interaction_ID`      | INT (PK) |Unique Interaction ID **`PRIMARY KEY`** |
| `user_ID` | INT (FK) | Links to `users.user_ID` |
| `query`| TEXT  | User Question|
| `response`| TEXT  | AI-Generated Response|
| `TIMESTAMP`|TIMESTAMP  |When the message was sent |

---
* **Purpose**:
  - Stores chatbot questions and responses.
---

* **Query Code**
```
query = '''
CREATE TABLE interactions (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    user_id UUID REFERENCES users(id) ON DELETE CASCADE,
    query TEXT NOT NULL,
    response TEXT NOT NULL,
    timestamp TIMESTAMP DEFAULT NOW()
);
'''
```




In [ ]:
chatbot_query = '''
CREATE TABLE IF NOT EXISTS interactions (
    interaction_ID CHAR(36) PRIMARY KEY, -- Storing UUID as CHAR(36)
    user_ID CHAR(36) NOT NULL, -- Ensuring consistency in foreign key type
    query TEXT NOT NULL,
    response TEXT NOT NULL,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP, -- Uses MySQL's timestamp handling
    FOREIGN KEY (user_ID) REFERENCES users(user_ID) ON DELETE CASCADE -- Enforcing valid user association
);
'''
execute_sql(chatbot_query, "interactions", db_conn)

⏳ Executing query for table: interactions...

✅ Success! Table 'interactions' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type     ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ interaction_ID       │ char(36) │  NO  │ PRI │ NULL              │ None              │
│ user_ID              │ char(36) │  NO  │ MUL │ NULL              │ None              │
│ query                │ text     │  NO  │     │ NULL              │ None              │
│ response             │ text     │  NO  │     │ NULL              │ None              │
│ timestamp            │ datetime │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
└──────────────────────┴──────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 12 : Course Reviews & Ratingss**  

| **Col_Name**|**Type** | **Description**|
|-------------|--------------------------|-----------------------------|
| `review_ID`      | INT (PK) |Unique Review ID **`PRIMARY KEY`** |
| `course_ID`| INT (FK)  | links to `courses_course_ID`|
| `user_ID` | INT (FK) | links to `users.user_ID` |
| `rating`| FLOAT  | 1-5 star rating|
| `review`| TEXT  | User Feedback|
| `created_at`|TIMESTAMP  |Creation Date |

---
* **Purpose**:
  - Allow students to rate courses (1️-5 stars) and leave reviews.
  - Helps new learners decide which courses are high-quality.
---

* **Query Code**
```
query = '''
CREATE TABLE course_reviews (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    user_id UUID REFERENCES users(id) ON DELETE CASCADE,
    course_id UUID REFERENCES courses(id) ON DELETE CASCADE,
    rating FLOAT CHECK (rating >= 1 AND rating <= 5),
    review TEXT,
    created_at TIMESTAMP DEFAULT NOW()
);
'''
```




In [ ]:
reviews_query = '''
CREATE TABLE IF NOT EXISTS course_reviews (
    review_ID CHAR(36) PRIMARY KEY,  -- Storing UUID as CHAR(36)
    user_ID CHAR(36) NOT NULL, -- Ensure consistency in foreign key type
    course_ID CHAR(36) NOT NULL, -- Ensure consistency in foreign key type
    rating REAL CHECK (rating >= 1.0 AND rating <= 5.0) NOT NULL, -- Ensuring valid rating range
    review TEXT DEFAULT NULL, -- Allowing optional reviews
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP, -- Automatic timestamp handling
    FOREIGN KEY (user_ID) REFERENCES users(user_ID) ON DELETE CASCADE, -- Enforcing valid user association
    FOREIGN KEY (course_ID) REFERENCES courses(course_ID) ON DELETE CASCADE -- Enforcing course linkage
);
'''
execute_sql(reviews_query, "course_reviews", db_conn)

⏳ Executing query for table: course_reviews...

✅ Success! Table 'course_reviews' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type     ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ review_ID            │ char(36) │  NO  │ PRI │ NULL              │ None              │
│ user_ID              │ char(36) │  NO  │ MUL │ NULL              │ None              │
│ course_ID            │ char(36) │  NO  │ MUL │ NULL              │ None              │
│ rating               │ double   │  NO  │     │ NULL              │ None              │
│ review               │ text     │ YES  │     │ NULL              │ None              │
│ created_at           │ datetime │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
└──────────────────────┴──────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 13 : Certificates & Badges**  


|**Column Name**|**Type**|**Description**|
|---------------|--------|---------------|
|`certificate_ID` |INT PK  | Certificate ID **`PRIMARY KEY`**|
|`user_id`        |INT (FK)|links to `users.user_ID`|
|`Course_id`      | INT (FK)| Links to `courses.course_ID`|
|`Certification_URL`|TEXT (URL)|Download Link|
|`Issued_at`|TIMESTAMP|When Issued|

|**Column Name**|**Type**|**Description**|
|---------------|--------|---------------|
|`badge_ID` |INT PK  | Badge ID **`PRIMARY KEY`**|
|`user_id`        |INT (FK)|links to `users.user_ID`|
|`badge_type`      | TEXT| `QUIZ MASTER`, `TOP Performer`|
|`earned_at`|TIMESTAMP|TIMESTAMP|



---
* **Purpose**:
  - Generate completion certificates for students.
  - Add achievement badges for milestones (e.g., "1️00% Course Completion", "Top Quiz Scores").
---

* **Query Code**
```
query = '''
CREATE TABLE certificates (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    user_id UUID REFERENCES users(id) ON DELETE CASCADE,
    course_id UUID REFERENCES courses(id) ON DELETE CASCADE,
    certificate_url TEXT NOT NULL,
    issued_at TIMESTAMP DEFAULT NOW()
);
'''
```




In [ ]:
certificate_query = '''
-- 🟢 Create the 'certificates' table with proper foreign keys
CREATE TABLE certificates (
    certificate_ID CHAR(36) PRIMARY KEY,
    user_ID CHAR(36) NOT NULL,  -- User ID as foreign key
    course_ID CHAR(36) NOT NULL, -- Course ID as foreign key
    certificate_url TEXT NOT NULL,
    issued_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_ID) REFERENCES users(user_ID) ON DELETE CASCADE,
    FOREIGN KEY (course_ID) REFERENCES courses(course_ID) ON DELETE CASCADE
);
'''

execute_sql(certificate_query, "certificates", db_conn)

⏳ Executing query for table: certificates...

✅ Success! Table 'certificates' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type     ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ certificate_ID       │ char(36) │  NO  │ PRI │ NULL              │ None              │
│ user_ID              │ char(36) │  NO  │ MUL │ NULL              │ None              │
│ course_ID            │ char(36) │  NO  │ MUL │ NULL              │ None              │
│ certificate_url      │ text     │  NO  │     │ NULL              │ None              │
│ issued_at            │ datetime │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
└──────────────────────┴──────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 14 : Payment System & Subscriptions (OPTIONAL)**  


|**Column Name**|**Type**|**Description**|
|---------------|--------|---------------|
|`payment_ID` |INT PK  | PAYMENT ID **`PRIMARY KEY`**|
|`user_ID`        |INT (FK)|links to `users.user_ID`|
|`course_ID`      | INT (FK)| Links to `courses.course_ID`|
|`subscription_Plan`|TEXT|`Monthly`, `Annual`|
|`amount`|FLOAT|Payment Amount|
|`payment_status`|TEXT|`Success`, `Pending`, `Failed`|
|`created_at`|TIMESTAMP|Timestamp

---

* **Query Code**
```
query = '''
CREATE TABLE payments (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    user_id UUID REFERENCES users(id) ON DELETE CASCADE,
    course_id UUID REFERENCES courses(id) ON DELETE CASCADE,
    subscription_plan TEXT CHECK (subscription_plan IN ('Monthly', 'Annual', 'One-Time')),
    amount FLOAT NOT NULL,
    payment_status TEXT CHECK (payment_status IN ('Success', 'Pending', 'Failed')) DEFAULT 'Pending',
    created_at TIMESTAMP DEFAULT NOW()
);
'''
```




In [ ]:
payment_query = '''
CREATE TABLE IF NOT EXISTS payments (
    payment_ID CHAR(36) PRIMARY KEY, -- Use CHAR(36) for UUIDs instead of randomblob
    user_ID CHAR(36) NOT NULL, -- Ensure consistency in foreign key type
    course_ID CHAR(36) NOT NULL, -- Ensure consistency in foreign key type
    subscription_plan TEXT CHECK (subscription_plan IN ('Monthly', 'Annual', 'One-Time')) NOT NULL, -- Plan must be specified
    amount DECIMAL(10,2) NOT NULL CHECK (amount >= 0), -- Ensuring valid positive payment amounts
    payment_status TEXT CHECK (payment_status IN ('Success', 'Pending', 'Failed')), -- Remove default value
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP, -- Auto-generated timestamp
    FOREIGN KEY (user_ID) REFERENCES users(user_ID) ON DELETE CASCADE, -- Enforcing valid user association
    FOREIGN KEY (course_ID) REFERENCES courses(course_ID) ON DELETE CASCADE -- Enforcing course linkage
);

'''
execute_sql(payment_query, "payments", db_conn)

⏳ Executing query for table: payments...

✅ Success! Table 'payments' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type          ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ payment_ID           │ char(36)      │  NO  │ PRI │ NULL              │ None              │
│ user_ID              │ char(36)      │  NO  │ MUL │ NULL              │ None              │
│ course_ID            │ char(36)      │  NO  │ MUL │ NULL              │ None              │
│ subscription_plan    │ text          │  NO  │     │ NULL              │ None              │
│ amount               │ decimal(10,2) │  NO  │     │ NULL              │ None              │
│ payment_status       │ text          │ YES  │     │ NULL              │ None              │
│ created_at           │ datetime      │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
└──────────────────────┴───────────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

## **Table 15 : `Password_Resets`**  


|**Column Name**|**Type**|**Description**|
|---------------|--------|---------------|
|`id` |INT PK  | RESET ID **`PRIMARY KEY`**|
|`email`        |VARCHAR(255)|links to `users.email`|
|`token`      | VARCHAR(255)| The token used to reset the mail|
|`created_at`|TIMESTAMP|creation time|
|`expires_at`|TIMESTAMP|expiration time|

---

* **Query Code**
```
query = '''
CREATE TABLE IF NOT EXISTS password_resets   (
    id INT AUTO_INCREMENT PRIMARY KEY,
    email VARCHAR(255) NOT NULL,
    token VARCHAR(255) NOT NULL UNIQUE,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    expires_at DATETIME NOT NULL,
    INDEX (email),
    FOREIGN KEY (email) REFERENCES CourseDataBase.users(email) ON DELETE CASCADE
);
'''
```




In [29]:
reset_password_query = '''
CREATE TABLE IF NOT EXISTS password_resets (
    id INT AUTO_INCREMENT PRIMARY KEY,
    email VARCHAR(255) NOT NULL,
    token VARCHAR(255) NOT NULL UNIQUE,
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
    expires_at DATETIME NOT NULL,
    INDEX (email),
    CONSTRAINT fk_email FOREIGN KEY (email) REFERENCES users(email) ON DELETE CASCADE
);
'''
execute_sql(reset_password_query, "password_resets", db_conn)

⏳ Executing query for table: password_resets...

✅ Success! Table 'password_resets' created or updated.

📌 Table Structure:

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Field                ┃ Type         ┃ Null ┃ Key ┃ Default           ┃ Extra             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ id                   │ int          │  NO  │ PRI │ NULL              │ auto_increment    │
│ email                │ varchar(255) │  NO  │ MUL │ NULL              │ None              │
│ token                │ varchar(255) │  NO  │ UNI │ NULL              │ None              │
│ created_at           │ datetime     │ YES  │     │ CURRENT_TIMESTAMP │ DEFAULT_GENERATED │
│ expires_at           │ datetime     │  NO  │     │ NULL              │ None              │
└──────────────────────┴──────────────┴──────┴─────┴───────────────────┴───────────────────┘

🔍 Query execution complete.

In [14]:
def visualize_db_schema(db_path):
    """
    🚀 Impressively visualizes the MySQL database schema with individual table visuals and an ER diagram.

    📌 Features:
    ✅ Displays each table separately in a well-formatted, rich table.
    ✅ Generates an individual image for each table with its fields.
    ✅ Creates an overall ER diagram showing relationships.
    ✅ Handles errors gracefully.

    🔥 Author: Basel Amr Barakat
    📧 Contact: baselamr52@gmail.com
    """
    console = Console()
    try:
        # 🔗 Connect to the database
        conn = mysql.connector.connect(
                    host=HOST,
                    user=USER,
                    password=PASSWORD,
                    database=DB_NAME
                )
        cursor = conn.cursor()

        # 🟢 Get list of tables from information_schema
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = %s;", (DB_NAME,))
        tables = [row[0] for row in cursor.fetchall()]

        if not tables:
            console.print("[bold red]❌ No tables found in the database![/bold red]")
            return

        # 🎭 Graphviz for ER diagram
        er_diagram = graphviz.Digraph(format='png', engine="dot")
        er_diagram.attr(rankdir='TB', bgcolor="white", fontname="Arial")

        # 🔍 Loop through each table
        for table in tables:
            # Get table columns info
            cursor.execute(f"DESCRIBE {table};")
            columns = cursor.fetchall()

            # Get foreign key relationships
            cursor.execute(f"""
                SELECT COLUMN_NAME, REFERENCED_TABLE_NAME
                FROM information_schema.key_column_usage
                WHERE TABLE_NAME = %s AND TABLE_SCHEMA = %s AND REFERENCED_TABLE_NAME IS NOT NULL;
            """, (table, DB_NAME))
            foreign_keys = cursor.fetchall()
            fk_map = {fk[0]: fk[1] for fk in foreign_keys}  # {column_name: referenced_table}

            # 🎨 Create an individual rich table for this table
            table_display = Table(title=f"📄 Table: {table}", box=box.DOUBLE, show_header=True, header_style="bold magenta")
            table_display.add_column("Column Name", style="green")
            table_display.add_column("Data Type", style="yellow", justify="center")
            table_display.add_column("Foreign Key", style="red", justify="center")

            # 🎨 Create a Graphviz visualization for this table
            table_graph = graphviz.Digraph(format='png', engine="dot")
            table_graph.attr(rankdir='TB', bgcolor="white", fontname="Arial")
            table_graph.node(table, shape="folder", style="filled", fillcolor="lightblue")

            # 📌 Add column details
            for col in columns:
                col_name, col_type = col[0], col[1]
                fk_reference = fk_map.get(col_name, "—")
                table_display.add_row(col_name, col_type, fk_reference if fk_reference != "—" else "[gray]—[/gray]")

                # 🏗️ Add columns to table Graphviz
                table_graph.node(f"{table}_{col_name}", label=f"{col_name}\n[{col_type}]", shape="box", style="filled", fillcolor="lightgray")
                table_graph.edge(table, f"{table}_{col_name}")

            # 🔗 Connect foreign key relationships in ER diagram
            if fk_map:
                for col, ref_table in fk_map.items():
                    er_diagram.edge(table, ref_table, label=f"🔗 {col}", color="red", penwidth="2.0")

            # 🖼️ Render and save table graph
            table_graph.render(f"tables/{table}_schema", format="png", cleanup=True)

            # 📊 Print the rich table in console
            console.print(table_display)

        # 🎨 Save and render ER diagram
        er_diagram.render("database_schema", format="png", cleanup=True)
        console.print("\n[bold green]✅ Schema visualization generated![/bold green] 🎉\n")
        console.print("[bold blue]📷 Each table saved as an image in the 'tables/' folder![/bold blue]")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        conn.close()


In [15]:
# 🎯 Usage
visualize_db_schema(DB_NAME)

           📄 Table: assignments           
╔═══════════════╦═══════════╦═════════════╗
║ Column Name   ║ Data Type ║ Foreign Key ║
╠═══════════════╬═══════════╬═════════════╣
║ assignment_ID ║ char(36)  ║      —      ║
║ course_ID     ║ char(36)  ║   courses   ║
║ title         ║   text    ║      —      ║
║ description   ║   text    ║      —      ║
║ deadline      ║ datetime  ║      —      ║
║ pass_grade    ║    int    ║      —      ║
║ created_at    ║ datetime  ║      —      ║
╚═══════════════╩═══════════╩═════════════╝

           📄 Table: certificates            
╔═════════════════╦═══════════╦═════════════╗
║ Column Name     ║ Data Type ║ Foreign Key ║
╠═════════════════╬═══════════╬═════════════╣
║ certificate_ID  ║ char(36)  ║      —      ║
║ user_ID         ║ char(36)  ║    users    ║
║ course_ID       ║ char(36)  ║   courses   ║
║ certificate_url ║   text    ║      —      ║
║ issued_at       ║ datetime  ║      —      ║
╚═════════════════╩═══════════╩═════════════╝

        📄 Table: chapter_content        
╔═════════════╦═══════════╦═════════════╗
║ Column Name ║ Data Type ║ Foreign Key ║
╠═════════════╬═══════════╬═════════════╣
║ chapter_ID  ║    int    ║  chapters   ║
║ content_ID  ║    int    ║   content   ║
║ position    ║    int    ║      —      ║
╚═════════════╩═══════════╩═════════════╝

             📄 Table: chapters             
╔═════════════╦══════════════╦═════════════╗
║ Column Name ║  Data Type   ║ Foreign Key ║
╠═════════════╬══════════════╬═════════════╣
║ chapter_ID  ║     int      ║      —      ║
║ title       ║ varchar(255) ║      —      ║
║ description ║     text     ║      —      ║
║ created_at  ║   datetime   ║      —      ║
╚═════════════╩══════════════╩═════════════╝

                               📄 Table: content                               
╔══════════════╦════════════════════════════════════════════════╦═════════════╗
║ Column Name  ║                   Data Type                    ║ Foreign Key ║
╠══════════════╬════════════════════════════════════════════════╬═════════════╣
║ content_ID   ║                      int                       ║      —      ║
║ content_type ║ enum('video','pdf','quiz','ppt','text','file') ║      —      ║
║ content_url  ║                  varchar(255)                  ║      —      ║
║ description  ║                      text                      ║      —      ║
║ duration     ║                     double                     ║      —      ║
║ created_at   ║                    datetime                    ║      —      ║
╚══════════════╩════════════════════════════════════════════════╩═════════════╝

...

       📄 Table: student_quiz_progress       
╔════════════════╦════════════╦═════════════╗
║ Column Name    ║ Data Type  ║ Foreign Key ║
╠════════════════╬════════════╬═════════════╣
║ progress_ID    ║    int     ║      —      ║
║ user_ID        ║  char(36)  ║    users    ║
║ quiz_ID        ║    int     ║   quizzes   ║
║ attempt_number ║    int     ║      —      ║
║ score          ║   float    ║      —      ║
║ passed         ║ tinyint(1) ║      —      ║
║ completed_at   ║  datetime  ║      —      ║
╚════════════════╩════════════╩═════════════╝

           📄 Table: submissions           
╔═══════════════╦═══════════╦═════════════╗
║ Column Name   ║ Data Type ║ Foreign Key ║
╠═══════════════╬═══════════╬═════════════╣
║ submission_ID ║ char(36)  ║      —      ║
║ user_ID       ║ char(36)  ║    users    ║
║ assignment_ID ║ char(36)  ║ assignments ║
║ file_url      ║   text    ║      —      ║
║ grade         ║   float   ║      —      ║
║ submitted_at  ║ datetime  ║      —      ║
║ attempts      ║    int    ║      —      ║
║ status        ║   text    ║      —      ║
╚═══════════════╩═══════════╩═════════════╝

                      📄 Table: users                       
╔═════════════════════════════╦══════════════╦═════════════╗
║ Column Name                 ║  Data Type   ║ Foreign Key ║
╠═════════════════════════════╬══════════════╬═════════════╣
║ user_ID                     ║   char(36)   ║      —      ║
║ full_name                   ║     text     ║      —      ║
║ email                       ║ varchar(255) ║      —      ║
║ password_hash               ║     text     ║      —      ║
║ role                        ║     text     ║      —      ║
║ bio                         ║     text     ║      —      ║
║ linkedin_url                ║     text     ║      —      ║
║ github_url                  ║     text     ║      —      ║
║ profile_picture             ║     text     ║      —      ║
║ preferences                 ║     json     ║      —      ║
║ education_level             ║     text     ║      —      ║
║ years_of_experience         ║     int      ║      —      ║
║ upload_cv_url               ║     text     ║      —      ║
║ preferred_learning_platform ║     text     ║      —      ║
║ location                    ║     text     ║      —      ║
║ available_for_projects      ║  tinyint(1)  ║      —      ║
║ programming_languages       ║     text     ║      —      ║
║ tools_and_technologies      ║     text     ║      —      ║
║ interests                   ║     text     ║      —      ║
║ preferred_project_types     ║     text     ║      —      ║
║ created_at                  ║  timestamp   ║      —      ║
║ updated_at                  ║  timestamp   ║      —      ║
║ email_verification_token    ║ varchar(255) ║      —      ║
║ verified                    ║  tinyint(1)  ║      —      ║
║ email_verification_expires  ║   datetime   ║      —      ║
╚═════════════════════════════╩══════════════╩═════════════╝

✅ Schema visualization generated! 🎉

📷 Each table saved as an image in the 'tables/' folder!

## **2.1 Draw Diagram**

In [16]:
pip install sqlalchemy mysql-connector-python sqlalchemy_schemadisplay Pillow

In [17]:
try:
  from eralchemy import render_er
except:
  !pip install eralchemy
  from eralchemy import render_er
  print("✅ eralchemy installed")

from IPython.display import Image
import mysql.connector
from sqlalchemy import create_engine, MetaData
from sqlalchemy_schemadisplay import create_schema_graph
from PIL import Image
import os
from PIL import Image
import io

✅ eralchemy installed


In [18]:
def generate_er_diagram(DB_NAME, HOST, USER, PASSWORD):
    """
    🚀 Generates and visualizes the ER diagram of the MySQL database schema.

    📌 Features:
    ✅ Connects to the database and analyzes the schema.
    ✅ Creates an ER diagram of the database schema using SQLAlchemy's schema graph.
    ✅ Displays the ER diagram using PIL.
    """
    try:
        # 🔗 Connect to the database using SQLAlchemy
        engine = create_engine(f'mysql+mysqlconnector://{USER}:{PASSWORD}@{HOST}/{DB_NAME}')

        # Create metadata and reflect the schema from the database
        metadata = MetaData()
        metadata.reflect(bind=engine)

        # 🏗️ Generate ER diagram using SQLAlchemy and Schema Display
        graph = create_schema_graph(
            engine=engine,  # Pass the engine as an argument
            metadata=metadata,
            show_datatypes=False,  # Hide datatypes for simplicity
            show_indexes=False,  # Hide indexes
            rankdir='LR',  # Left to right orientation
            concentrate=False  # Do not concentrate edges
        )

        # 🖼️ Save the diagram to a binary stream instead of a file
        image_stream = io.BytesIO()
        graph.write_png(image_stream)
        image_stream.seek(0)  # Reset the stream position to the start

        # 📷 Display the ER diagram directly from the stream
        #img = Image.open(image_stream)
        #img.show()

        print(f"\n✅ ER Diagram generated and displayed!")

    except Exception as e:
        print(f"❌ Error: {e}")


In [19]:
generate_er_diagram(DB_NAME=DB_NAME, HOST=HOST, USER=USER, PASSWORD=PASSWORD)

❌ Error: expected str, bytes or os.PathLike object, not BytesIO


## **2.2 : Verify Table Structure**

In [20]:
def visualize_tables(DB_NAME=DB_NAME):
    """
    🚀 Visually presents each table's structure in an elegant and interactive format.

    📌 Features:
    ✅ Displays each table in a stylish format.
    ✅ Generates graphical schema images for each table and saves them in the 'structure' folder.
    ✅ Uses rich formatting for enhanced readability.

    🔥 Author: Basel Amr Barakat
    📧 Contact: baselamr52@gmail.com
    """
    console = Console()

    # 📂 Ensure 'structure' folder exists
    output_folder = "structure"
    os.makedirs(output_folder, exist_ok=True)

    try:
        # 🎯 Connect to the MySQL database
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # 🎯 Get all table names from MySQL
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = DATABASE();")
        tables = [row[0] for row in cursor.fetchall()]

        if not tables:
            console.print("[bold red]❌ No tables found in the database![/bold red]")
            return

        for table in tables:
            console.print(Panel(f"[bold cyan]📂 Table:[/bold cyan] [yellow]{table}[/yellow]", style="bold blue", expand=False))

            # 📊 Get table structure
            cursor.execute(f"DESCRIBE {table}")
            table_info = cursor.fetchall()

            # 🎨 Create a stylish rich table
            rich_table = RichTable(title=f"📜 Structure of {table}", box=box.DOUBLE, header_style="bold magenta")
            rich_table.add_column("Column Name", style="green")
            rich_table.add_column("Data Type", style="yellow", justify="center")
            rich_table.add_column("Null", style="red", justify="center")
            rich_table.add_column("Key", style="bold white", justify="center")
            rich_table.add_column("Default", style="blue", justify="center")
            rich_table.add_column("Extra", style="cyan", justify="center")

            # 🎭 Graphviz for visual representation
            diagram = graphviz.Digraph(format='png', engine="dot")
            diagram.attr(rankdir='TB', bgcolor="white")
            diagram.node(table, shape="box", style="filled", fillcolor="lightblue", fontname="Arial")

            # 🛠 Populate table and diagram
            for row in table_info:
                col_name, col_type, is_null, key, default_val, extra = row
                default_val = default_val if default_val is not None else "—"
                is_null_status = "✅" if is_null == "YES" else "—"
                key_status = "🔑" if key else "—"

                rich_table.add_row(col_name, col_type, is_null_status, key_status, str(default_val), extra)

                # 🖼️ Add field to diagram
                diagram.node(col_name, shape="ellipse", style="filled", fillcolor="lightgray", fontname="Arial")
                diagram.edge(table, col_name, color="black")

            # 🖥️ Print the table structure
            console.print(rich_table)
            console.print("-" * 80)

            # 📷 Generate and save the table structure image in 'structure' folder
            image_path = os.path.join(output_folder, f"{table}_structure")
            diagram.render(image_path, format="png", cleanup=True)
            console.print(f"[bold green]✅ Table structure image saved in '{output_folder}/{table}_structure.png'[/bold green] 🎉\n")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")
    finally:
        cursor.close()
        conn.close()


In [36]:
visualize_tables(DB_NAME)

╭───────────────────────╮
│ 📂 Table: assignments │
╰───────────────────────╯

                           📜 Structure of assignments                            
╔═══════════════╦═══════════╦══════╦═════╦═══════════════════╦═══════════════════╗
║ Column Name   ║ Data Type ║ Null ║ Key ║      Default      ║       Extra       ║
╠═══════════════╬═══════════╬══════╬═════╬═══════════════════╬═══════════════════╣
║ assignment_ID ║ char(36)  ║  —   ║ 🔑  ║      uuid()       ║ DEFAULT_GENERATED ║
║ course_ID     ║ char(36)  ║  —   ║ 🔑  ║         —         ║                   ║
║ title         ║   text    ║  —   ║  —  ║         —         ║                   ║
║ description   ║   text    ║  —   ║  —  ║         —         ║                   ║
║ deadline      ║ datetime  ║  ✅  ║  —  ║         —         ║                   ║
║ pass_grade    ║    int    ║  —   ║  —  ║         —         ║                   ║
║ created_at    ║ datetime  ║  ✅  ║  —  ║ CURRENT_TIMESTAMP ║ DEFAULT_GENERATED ║
╚═══════════════╩═══════════╩══════╩═════╩═══════════════════╩═══════════════════╝

--------------------------------------------------------------------------------

✅ Table structure image saved in 'structure/assignments_structure.png' 🎉

╭────────────────────────╮
│ 📂 Table: certificates │
╰────────────────────────╯

...

✅ Table structure image saved in 'structure/submissions_structure.png' 🎉

╭─────────────────╮
│ 📂 Table: users │
╰─────────────────╯

                                       📜 Structure of users                                       
╔═════════════════════════════╦══════════════╦══════╦═════╦═══════════════════╦═══════════════════╗
║ Column Name                 ║  Data Type   ║ Null ║ Key ║      Default      ║       Extra       ║
╠═════════════════════════════╬══════════════╬══════╬═════╬═══════════════════╬═══════════════════╣
║ user_ID                     ║   char(36)   ║  —   ║ 🔑  ║      uuid()       ║ DEFAULT_GENERATED ║
║ full_name                   ║     text     ║  —   ║  —  ║         —         ║                   ║
║ email                       ║ varchar(255) ║  —   ║ 🔑  ║         —         ║                   ║
║ password_hash               ║     text     ║  —   ║  —  ║         —         ║                   ║
║ role                        ║     text     ║  —   ║  —  ║         —         ║                   ║
║ bio                         ║     text     ║  ✅  ║  —  ║         —         ║                   ║
║ linkedin_url                ║     text     ║  ✅  ║  —  ║         —         ║                   ║
║ github_url                  ║     text     ║  ✅  ║  —  ║         —         ║                   ║
║ profile_picture             ║     text     ║  ✅  ║  —  ║         —         ║                   ║
║ preferences                 ║     json     ║  ✅  ║  —  ║         —         ║                   ║
║ education_level             ║     text     ║  ✅  ║  —  ║         —         ║                   ║
║ years_of_experience         ║     int      ║  ✅  ║  —  ║         —         ║                   ║
║ upload_cv_url               ║     text     ║  ✅  ║  —  ║         —         ║                   ║
║ preferred_learning_platform ║     text     ║  ✅  ║  —  ║         —         ║                   ║
║ location                    ║     text     ║  ✅  ║  —  ║         —         ║                   ║
║ available_for_projects      ║  tinyint(1)  ║  ✅  ║  —  ║         —         ║                   ║
║ programming_languages       ║     text     ║  ✅  ║  —  ║         —         ║                   ║
║ tools_and_technologies      ║     text     ║  ✅  ║  —  ║         —         ║                   ║
║ interests                   ║     text     ║  ✅  ║  —  ║         —         ║                   ║
║ preferred_project_types     ║     text     ║  ✅  ║  —  ║         —         ║                   ║
║ created_at                  ║  timestamp   ║  ✅  ║  —  ║ CURRENT_TIMESTAMP ║ DEFAULT_GENERATED ║
║ updated_at                  ║  timestamp   ║  ✅  ║  —  ║ CURRENT_TIMESTAMP ║ DEFAULT_GENERATED ║
║ email_verification_token    ║ varchar(255) ║  ✅  ║  —  ║         —         ║                   ║
║ verified                    ║  tinyint(1)  ║  ✅  ║  —  ║         0         ║                   ║
║ email_verification_expires  ║   datetime   ║  ✅  ║  —  ║         —         ║                   ║
╚═════════════════════════════╩══════════════╩══════╩═════╩═══════════════════╩═══════════════════╝

--------------------------------------------------------------------------------

✅ Table structure image saved in 'structure/users_structure.png' 🎉

In [21]:
def inspect_database(database_name, host, user, password):
    # 🎯 Connect to the MySQL database
    conn = connect_to_mysql(host, user, password, database_name)
    if not conn:
        return

    cursor = conn.cursor()

    try:
        console = Console()

        # 📋 Get the list of tables in the database
        cursor.execute(f"SHOW TABLES;")
        tables = cursor.fetchall()

        # Create a table to display database information
        summary_table = Table(show_header=True, header_style="bold cyan", box=box.ROUNDED)
        summary_table.add_column("Table Name", style="bold magenta")
        summary_table.add_column("Number of Attributes", style="bold green")
        summary_table.add_column("Number of Rows", style="bold yellow")

        for table in tables:
            table_name = table[0]

            # 📊 Get attributes and count them
            cursor.execute(f"DESCRIBE {table_name};")
            attributes = cursor.fetchall()
            num_attributes = len(attributes)

            # 📊 Get the number of rows in the table
            cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
            num_rows = cursor.fetchone()[0]

            # Add table summary to the summary table
            summary_table.add_row(table_name, str(num_attributes), str(num_rows))

            # 🎯 Get 5 samples from the table (if available)
            cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
            samples = cursor.fetchall()

            # Display table information in a professional way
            console.print(f"\n🛠️  Inspecting table: [bold blue]{table_name}[/bold blue]", style="bold yellow")

            # Display column names and types
            column_table = Table(show_header=True, header_style="bold cyan", box=box.SIMPLE)
            column_table.add_column("Attribute Name", style="bold magenta")
            column_table.add_column("Data Type", style="bold green")
            for attribute in attributes:
                column_table.add_row(attribute[0], attribute[1])
            console.print(column_table)

            # Display samples from the table
            if samples:
                console.print(f"\n📦  The Table has data[bold blue]{table_name}[/bold blue]:", style="bold yellow")
                #sample_table = Table(show_header=True, header_style="bold cyan", box=box.SIMPLE)
                #for col in range(len(attributes)):
                #    sample_table.add_column(f"Sample {col + 1}", style="bold green")
                #for sample in samples:
                #    sample_table.add_row(*[str(value) for value in sample])
                #console.print(sample_table)
            else:
                console.print(f"[bold red]❌ No samples available in this table.[/bold red]")

        # Display the database summary table
        console.print("\n📊  Database Summary:", style="bold yellow")
        console.print(summary_table)

    except mysql.connector.Error as err:
        console.print(f"❌ Error: {err}", style="bold red")

    finally:
        conn.close()

In [38]:
inspect_database(DB_NAME, HOST, USER, PASSWORD)

✅ Successfully connected to the MySQL database!

🛠️  Inspecting table: assignments

 Attribute Name   Data Type  
 ──────────────────────────── 
  assignment_ID    char(36)   
  course_ID        char(36)   
  title            text       
  description      text       
  deadline         datetime   
  pass_grade       int        
  created_at       datetime  

❌ No samples available in this table.

🛠️  Inspecting table: certificates

...

🛠️  Inspecting table: users

 Attribute Name                Data Type     
 ──────────────────────────────────────────── 
  user_ID                       char(36)      
  full_name                     text          
  email                         varchar(255)  
  password_hash                 text          
  role                          text          
  bio                           text          
  linkedin_url                  text          
  github_url                    text          
  profile_picture               text          
  preferences                   json          
  education_level               text          
  years_of_experience           int           
  upload_cv_url                 text          
  preferred_learning_platform   text          
  location                      text          
  available_for_projects        tinyint(1)    
  programming_languages         text          
  tools_and_technologies        text          
  interests                     text          
  preferred_project_types       text          
  created_at                    timestamp     
  updated_at                    timestamp     
  email_verification_token      varchar(255)  
  verified                      tinyint(1)    
  email_verification_expires    datetime     

📦  The Table has datausers:

📊  Database Summary:

╭───────────────────────┬──────────────────────┬────────────────╮
│ Table Name            │ Number of Attributes │ Number of Rows │
├───────────────────────┼──────────────────────┼────────────────┤
│ assignments           │ 7                    │ 0              │
│ certificates          │ 5                    │ 0              │
│ chapter_content       │ 3                    │ 0              │
│ chapters              │ 4                    │ 0              │
│ content               │ 6                    │ 0              │
│ course_chapters       │ 3                    │ 0              │
│ course_reviews        │ 6                    │ 0              │
│ courses               │ 13                   │ 0              │
│ enrollments           │ 6                    │ 0              │
│ interactions          │ 5                    │ 0              │
│ password_resets       │ 5                    │ 0              │
│ payments              │ 7                    │ 0              │
│ qa                    │ 7                    │ 0              │
│ questions             │ 7                    │ 0              │
│ quizzes               │ 7                    │ 0              │
│ student_progress      │ 7                    │ 0              │
│ student_quiz_progress │ 7                    │ 0              │
│ submissions           │ 8                    │ 0              │
│ users                 │ 25                   │ 1              │
╰───────────────────────┴──────────────────────┴────────────────╯

# **3 Insert Some Data to the tables**

In [22]:
def display_table_sample(table_name, num_samples=5):
    """
    📌 Displays a visually impressive sample of a table from a MySQL database.

    ✅ Uses Rich for a stunning table visualization.
    ✅ Falls back to Tabulate for larger samples.
    ✅ Shows column headers, formatted data, and handles empty tables gracefully.
    """
    console = Console()

    try:
        # 🎯 Connect to MySQL using SQLAlchemy
        engine = create_engine(f"mysql+mysqlconnector://{USER}:{PASSWORD}@{HOST}/{DB_NAME}")

        # 🔍 Query Data (Use ORDER BY RAND() for random sampling in MySQL)
        query = f"SELECT * FROM {table_name} ORDER BY RAND() LIMIT {num_samples};"
        df = pd.read_sql(query, engine)

        if df.empty:
            console.print(f"[bold red]❌ No data found in table '{table_name}'![/bold red]")
            return

        # 🎨 Display using Rich if sample size is small
        if num_samples <= 10:
            table = Table(title=f"📊 Sample Data from '{table_name}'", box=box.DOUBLE, highlight=True)
            for col in df.columns:
                table.add_column(col, style="cyan", justify="center")

            for _, row in df.iterrows():
                table.add_row(*[str(value) for value in row])

            console.print(table)

        # 📜 If more than 10 samples, use Tabulate for readability
        else:
            console.print(f"\n📌 Showing {num_samples} samples from '{table_name}':\n")
            print(tabulate(df, headers="keys", tablefmt="fancy_grid"))

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

In [23]:
# 🎨 Custom rich theme for stylish console output
custom_theme = Theme({
    "title": "bold magenta",
    "info": "bold cyan",
    "data": "bold yellow",
    "error": "bold red"
})
console = Console(theme=custom_theme)

def visualize_dataset(csv_path):
    """
    🔥 Impressively Visualizes a Dataset with:
    ✅ Unique values in categorical columns
    ✅ Column names with rich formatting
    ✅ Dark-themed interactive visualizations using Plotly

    📧 Author: Basel Amr Barakat
    """
    try:
        # 🚀 Load dataset
        df = pd.read_csv(csv_path)

        # 🎭 Identify categorical and numerical columns
        cat_cols = df.select_dtypes(include=['object']).columns
        num_cols = df.select_dtypes(include=['number']).columns

        # 🎨 Title in the middle
        console.print(Panel("🔥 [bold magenta] Dataset Overview [/bold magenta] 🔥", style="bold blue"))

        # 📂 Display Column Names
        table = Table(title="📋 Column Names", header_style="bold yellow", box=SIMPLE)
        table.add_column("Index", style="cyan", justify="center")
        table.add_column("Column Name", style="bold white", justify="left")
        for i, col in enumerate(df.columns):
            table.add_row(str(i), col)
        console.print(table)

        # 🎭 Show Unique Values for Categorical Columns
        if len(cat_cols) > 0:
            console.print(Panel("🎭 [bold cyan] Unique Values in Categorical Columns [/bold cyan]", style="bold blue"))
            for col in cat_cols:
                console.print(f"📌 [bold cyan]{col}[/bold cyan]: {df[col].nunique()} unique values")
                console.print(f"📝 [yellow]{df[col].unique()[:10]}...[/yellow]")  # Show first 10 unique values
            console.print("-" * 80)

        # 🔢 Show Statistical Summary for Numerical Data
        console.print(Panel("📊 [bold green] Numerical Data Summary [/bold green]", style="bold blue"))
        console.print(df.describe())

        # 🎨 Dark Theme Visualizations with Plotly
        if len(num_cols) > 0:
            fig = px.histogram(df, x=num_cols[0],
                               title=f"🔥 Distribution of {num_cols[0]} 🔥",
                               template="plotly_dark",
                               color_discrete_sequence=["cyan"])
            fig.update_layout(title_x=0.5)  # Center the title
            fig.show()
        else:
            console.print("[bold red]❌ No numerical columns found for visualization![/bold red]")

    except Exception as e:
        console.print(f"[error]❌ Error:[/error] {e}")

In [24]:
# 🛠️ Usage Example:
visualize_dataset("udemy_course_data.csv")

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔥  Dataset Overview  🔥                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

        📋 Column Names        
                               
  Index   Column Name          
 ───────────────────────────── 
    0     course_id            
    1     course_title         
    2     url                  
    3     is_paid              
    4     price                
    5     num_subscribers      
    6     num_reviews          
    7     num_lectures         
    8     level                
    9     content_duration     
   10     published_timestamp  
   11     subject              
   12     profit               
   13     published_date       
   14     published_time       
   15     year                 
   16     month                
   17     day                 

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🎭  Unique Values in Categorical Columns                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

📌 course_title: 3668 unique values

📝 ['Ultimate Investment Banking Course'
 'Complete GST Course & Certification - Grow Your CA Practice'
 'Financial Modeling for Business Analysts and Consultants'
 'Beginner to Pro - Financial Analysis in Excel 2017'
 'How To Maximize Your Profits Trading Options'
 'Trading Penny Stocks: A Guide for All Levels In 2017'
 'Investing And Trading For Beginners: Mastering Price Charts'
 'Trading Stock Chart Patterns For Immediate, Explosive Gains'
 'Options Trading 3 : Advanced Stock Profit and Success Method'
 'The Only Investment Strategy You Need For Your Retirement']...

...

📌 published_time: 3552 unique values

📝 ['20:58:58Z' '16:34:20Z' '19:26:30Z' '20:07:24Z' '14:57:18Z' '15:13:30Z'
 '18:23:12Z' '22:13:03Z' '00:14:03Z' '18:13:32Z']...

--------------------------------------------------------------------------------

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 📊  Numerical Data Summary                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

course_id        price  num_subscribers   num_reviews  num_lectures  \
count  3.683000e+03  3683.000000      3683.000000   3683.000000   3683.000000   
mean   6.764546e+05    65.992398      3193.371165    156.448004     40.062178   
std    3.437217e+05    60.985586      9498.231406    935.078241     50.366788   
min    8.324000e+03     0.000000         0.000000      0.000000      0.000000   
25%    4.077270e+05    20.000000       110.000000      4.000000     15.000000   
50%    6.882440e+05    45.000000       911.000000     18.000000     25.000000   
75%    9.617290e+05    95.000000      2537.500000     67.000000     45.000000   
max    1.282064e+06   200.000000    268923.000000  27445.000000    779.000000   

             profit         year        month          day  
count  3.683000e+03  3683.000000  3683.000000  3683.000000  
mean   2.402885e+05  2015.433342     6.162639    15.841162  
std    1.000760e+06     1.185920     3.379314     8.780906  
min    0.000000e+00  2011.000000     1.000000     1.000000  
25%    1.567500e+03  2015.000000     3.000000     8.000000  
50%    2.305000e+04  2016.000000     6.000000    16.000000  
75%    1.182600e+05  2016.000000     9.000000    23.000000  
max    2.431680e+07  2017.000000    12.000000    31.000000

## **3.1 Insert Data into course table**

### **3.1.1 Load Excel data**

In [25]:
file_path = "udemy_course_data.csv"
df = pd.read_csv(file_path)

### **3.1.2 Rename columns to match the table**

In [26]:
# Rename columns to match the table
df.rename(columns={
    "course_title": "course_title",
    "subject": "category",
    "level": "level",
    "price": "price",
    "content_duration": "total_hours",
    "published_timestamp": "created_at"
}, inplace=True)

In [27]:
# Normalize the 'level' column
level_mapping = {
    "Beginner Level": "Beginner",
    "Intermediate Level": "Intermediate",
    "Expert Level": "Advanced",
    "All Levels": "Beginner"  # Assigning a default category
}
df["level"] = df["level"].map(level_mapping).fillna("Beginner")

In [28]:
# Convert 'total_hours' to float
def convert_duration(duration):
    if "hour" in duration:
        hours = re.findall(r"[\d.]+", duration)
        return float(hours[0]) if hours else 0
    elif "min" in duration:
        mins = re.findall(r"[\d.]+", duration)
        return float(mins[0]) / 60 if mins else 0
    return 0

df["total_hours"] = df["total_hours"].astype(str).apply(convert_duration)

In [29]:
# Add required columns
df["description"] = "No description available"
df["thumbnail_url"] = None
df["updated_at"] = df["created_at"]

In [30]:
# Select only relevant columns
df = df[["course_title", "description", "category", "level", "price", "thumbnail_url", "total_hours", "created_at", "updated_at"]]

### **3.1.1 Generate Course ID**

In [31]:
def generate_course_id():
    """Generate a unique 32-character hexadecimal course ID."""
    return ''.join(random.choices(string.hexdigits.lower(), k=32))

### **3.1.2 Convert Duration**

In [32]:
def convert_duration(duration):
    """Convert duration from 'X hours' or 'X mins' to float (total hours)."""
    if pd.isna(duration):
        return 0.0
    try:
        if "hour" in duration:
            return float(duration.split()[0])
        elif "min" in duration:
            return round(float(duration.split()[0]) / 60, 2)
    except:
        return 0.0
    return 0.0

### **3.1.3 map level**

In [33]:
def map_level(level):
    """Map CSV levels to database levels (Beginner, Intermediate, Advanced)."""
    level_mapping = {
        "Beginner Level": "Beginner",
        "Intermediate Level": "Intermediate",
        "Expert Level": "Advanced",
        "All Levels": "Intermediate"  # Assuming 'All Levels' maps to 'Intermediate'
    }
    return level_mapping.get(level, "Beginner")  # Default to Beginner if unknown

### **3.1.4 Insert Actual data**

In [34]:
from transformers import pipeline
import warnings
from huggingface_hub import hf_hub_download

# Suppress the warning for setting pad_token_id
warnings.filterwarnings('ignore', message="Setting pad_token_id to eos_token_id")

In [35]:
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer

def generate_course_details(course_title):
    """
    Generate professional course details similar to platforms like Udemy without repeating the question in the response.

    Parameters:
        course_title (str): The title of the course for which to generate details.

    Returns:
        dict: A dictionary containing:
            - description: The course description.
            - what_you_will_learn: A list of key learnings from the course.
            - who_this_course_is_for: The target audience for the course.
    """
    # Initialize model and tokenizer
    model = GPT2LMHeadModel.from_pretrained("gpt2")
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

    # Explicitly set pad_token_id to eos_token_id
    model.config.pad_token_id = model.config.eos_token_id

    # Pass `pad_token_id` explicitly in the pipeline
    generator = pipeline('text-generation', model=model, tokenizer=tokenizer, pad_token_id=model.config.eos_token_id)

    # Refined prompts to match the desired tone and content
    description_prompt = (
        f"Write a detailed, professional course description for '{course_title}' similar to courses on Udemy. "
        "The description should emphasize the key benefits, practical application, and provide a clear reason why "
        "this course is valuable for learners. Use language that is direct, engaging, and professional."
    )

    learning_prompt = (
        f"List the key skills and concepts students will learn in a course titled '{course_title}' on an online platform like Udemy. "
        "The skills should be actionable and clearly defined. Include practical examples and the most essential tools for success."
    )

    audience_prompt = (
        f"Describe the ideal target audience for the course '{course_title}'. The audience description should be clear, "
        "engaging, and indicate the background or expertise needed for learners to benefit from the course."
    )

    try:
        # Generate Course Description
        description_response = generator(description_prompt, max_length=300, num_return_sequences=1, truncation=True)
        description = description_response[0]['generated_text'].strip()

        # Generate 'What You Will Learn'
        learning_response = generator(learning_prompt, max_length=200, num_return_sequences=1, truncation=True)
        learning = learning_response[0]['generated_text'].strip()

        # Generate 'Who This Course Is For'
        audience_response = generator(audience_prompt, max_length=200, num_return_sequences=1, truncation=True)
        audience = audience_response[0]['generated_text'].strip()

        # Return the details as a dictionary
        return {
            'description': description,
            'what_you_will_learn': learning,
            'who_this_course_is_for': audience
        }

    except Exception as e:
        print(f"Error generating course details: {e}")
        return None


In [36]:
course_title = "Python Programming for Beginners"
course_details = generate_course_details(course_title)

if course_details:
    # Print the course details with rich formatting
    console.print("[bold magenta]Course Title:[/bold magenta]", course_title)
    console.print("\n[bold cyan]Course Description:[/bold cyan]")
    console.print(Text(course_details['description'], style="dim"))

    console.print("\n[bold cyan]What You Will Learn:[/bold cyan]")
    console.print(Text(course_details['what_you_will_learn'], style="dim"))

    console.print("\n[bold cyan]Who This Course Is For:[/bold cyan]")
    console.print(Text(course_details['who_this_course_is_for'], style="dim"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

Write a detailed, professional course description for 'Python Programming for Beginners' similar to courses on 
Udemy. The description should emphasize the key benefits, practical application, and provide a clear reason why 
this course is valuable for learners. Use language that is direct, engaging, and professional. Include an 
explanation of any specific feature described. Include a description of any basic or functional concepts. Create a 
project to test it, or to provide a demonstration of usage of some feature, or even discuss the program. Include a 
detailed report on the progress you were making. Provide a brief history of your application. Include a detailed 
written discussion of your reasoning, your approach, and others. Consider whether a course is suitable for other 
learning environments and topics. The following sections help you take each one as close to a beginner's course as 
possible by highlighting the strengths and weaknesses of each course.

Python Python Programming, the best and most complete programming language for developing Python applications, 
offers complete support for many programming language features in Python. An easy and simple introduction to the 
language provides excellent support for learning and coding in Python. Additionally, it allows easy configuration 
of applications and tools; the easy integration with other languages, applications, and frameworks. One of the most
valuable features in other programming languages is the ability to write programs quickly using a number of 
different writing algorithms. Programming is the ultimate way for developers to learn programming language, but 
there are pitfalls to ensure the right kind of programming language is chosen. Some programming languages are

What You Will Learn:

List the key skills and concepts students will learn in a course titled 'Python Programming for Beginners' on an 
online platform like Udemy. The skills should be actionable and clearly defined. Include practical examples and the
most essential tools for success.

It's also important to keep your skills concise. Be very clear if you want students to remember what they've 
learned or why.

Use clear explanations where appropriate.

The primary purpose of lessons is to help students to evaluate concepts and solutions.

Using an effective method should be central to learning in Python, so you focus on not only the principles but also
the knowledge as well; you'll be learning everything. Learn the concepts and code that you need to use for your 
projects, for example. Be ready to learn by being on the same page when you write code.

To increase your productivity in Python:

1) Understand Python in general

There are many methods for learning Python based on your own experience

Who This Course Is For:

Describe the ideal target audience for the course 'Python Programming for Beginners'. The audience description 
should be clear, engaging, and indicate the background or expertise needed for learners to benefit from the course.
For beginners, it should cover basic programming syntax, basic languages, idioms, the basics of Python, etc. for 
advanced students, and explanations of some of the major features of the course. In addition, it should cover the 
fundamentals of reading and learning Python while the course is taught. However, for advanced readers, it should be
explained very clearly by the point person who will help the student understand the course quickly through a clear,
interesting, and easy-to-understand context of the topic. Topics should be arranged in the appropriate order, such 
as:

How to think about programming in Python

Understanding programming language syntax

Languages to use. Specifically, they should outline the basic concepts of Python: syntax, classes, classes, 
methods, classes, types, methods,

In [37]:
import random

def insert_actual_data(db_path, csv_path, table_name, num_courses=150):
    """
    📌 Inserts a specified number of courses from a CSV file into a MySQL database.

    ✅ Automatically maps CSV columns to database columns.
    ✅ Deletes existing records before inserting new ones.
    ✅ Handles data type conversions gracefully.
    ✅ Uses default values for missing columns.
    ✅ Uses dynamically generated descriptions and course information.
    ✅ Allows limiting the number of inserted courses.
    """
    console = Console()

    try:
        # 🎯 Connect to the MySQL database
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # 📂 Load CSV into Pandas
        df = pd.read_csv(csv_path)

        if df.empty:
            console.print("[bold red]❌ The dataset is empty![/bold red]")
            return

        # Limit the dataset to the specified number of courses
        df = df.head(num_courses)

        # 🔍 Check if the table exists in MySQL
        table_check_query = f"""
            SELECT COUNT(*)
            FROM information_schema.tables
            WHERE table_schema = '{DB_NAME}'
            AND table_name = '{table_name}';
        """
        cursor.execute(table_check_query)
        table_exists = cursor.fetchone()[0]

        if not table_exists:
            console.print(f"[bold red]❌ Table '{table_name}' does not exist![/bold red]")
            return

        # 🎯 Fetch all instructors (user_IDs)
        cursor.execute("SELECT user_ID FROM users WHERE role = 'Instructor';")  # Assuming role indicates instructors
        instructors = cursor.fetchall()
        instructor_list = [instructor[0] for instructor in instructors]  # Extract user_IDs of instructors

        if not instructor_list:
            console.print("[bold red]❌ No instructors found in the database![/bold red]")
            return

        # 🎯 Transform Necessary Columns
        df_transformed = pd.DataFrame()

        # Generate course IDs
        df_transformed["course_ID"] = [generate_course_id() for _ in range(len(df))]

        # Map CSV columns to database columns
        df_transformed["course_title"] = df["course_title"]
        df_transformed["category"] = df["subject"]
        df_transformed["level"] = df["level"].apply(map_level)  # Assuming map_level is defined
        df_transformed["instructor_id"] = [random.choice(instructor_list) for _ in range(len(df))]  # Randomly assign instructors
        df_transformed["price"] = df["price"].astype(float)
        df_transformed["thumbnail_url"] = df["url"]
        df_transformed["total_hours"] = df["content_duration"].apply(convert_duration)

        # 🛠 Generate additional course details
        console.print("[bold cyan]⏳ Generating course details...[/bold cyan]")
        course_details = [generate_course_details(title) for title in tqdm(df["course_title"], desc="Processing Courses", unit="course")]

        df_transformed["description"] = [details["description"] for details in course_details]
        df_transformed["what_you_will_learn"] = [details["what_you_will_learn"] for details in course_details]
        df_transformed["who_this_course_is_for"] = [details["who_this_course_is_for"] for details in course_details]

        # Handle missing values
        df_transformed.fillna('', inplace=True)

        # Handle timestamps
        df_transformed["created_at"] = df["published_date"].apply(
            lambda x: x + " 00:00:00" if pd.notna(x) else datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        df_transformed["updated_at"] = df_transformed["created_at"]

        # 🛠 Delete existing records
        delete_query = f"DELETE FROM {table_name};"
        cursor.execute(delete_query)
        conn.commit()
        console.print(f"[bold yellow]⚠️ Deleted existing records from '{table_name}' table.[/bold yellow]")

        # 🛠 Insert data into the table
        console.print("[bold cyan]⏳ Inserting data into the database...[/bold cyan]")
        insert_query = f"""
            INSERT INTO {table_name} (
                course_ID, course_title, category, level, instructor_id, price,
                thumbnail_url, total_hours, description, what_you_will_learn,
                who_this_course_is_for, created_at, updated_at
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """

        for _, row in tqdm(df_transformed.iterrows(), total=len(df_transformed), desc="Inserting Courses", unit="course", leave=False, dynamic_ncols=True):
            cursor.execute(insert_query, tuple(row))

        conn.commit()
        console.print(f"[bold green]✅ Successfully inserted {len(df_transformed)} rows into '{table_name}'![/bold green] 🎉")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close


In [49]:
table_name = "courses"
insert_actual_data(DB_NAME, file_path, table_name,15)

⏳ Generating course details...

Processing Courses: 100%|██████████| 15/15 [01:46<00:00,  7.07s/course]


⚠️ Deleted existing records from 'courses' table.

⏳ Inserting data into the database...

✅ Successfully inserted 15 rows into 'courses'! 🎉

In [50]:
display_table_sample(table_name, 5)

                                           📊 Sample Data from 'courses'                                           
╔════════╦════════╦════════╦════════╦════════╦════════╦═══════╦════════╦════════╦═══════╦════════╦═══════╦════════╗
║ cours… ║ cours… ║ descr… ║ categ… ║ level  ║ instr… ║ price ║ thumb… ║ total… ║ crea… ║ updat… ║ what… ║ who_t… ║
╠════════╬════════╬════════╬════════╬════════╬════════╬═══════╬════════╬════════╬═══════╬════════╬═══════╬════════╣
║ dde8c… ║ Finan… ║ Write  ║ Busin… ║ Inter… ║ eaa3d… ║ 30.0  ║ https… ║  1.5   ║ 2016… ║ 2016-… ║ List  ║ Descr… ║
║        ║ Manag… ║   a    ║ Finan… ║        ║        ║       ║        ║        ║ 00:0… ║ 00:00… ║  the  ║  the   ║
║        ║  Risk  ║ detai… ║        ║        ║        ║       ║        ║        ║       ║        ║  key  ║ ideal  ║
║        ║  and   ║ profe… ║        ║        ║        ║       ║        ║        ║       ║        ║ skil… ║ target ║
║        ║ Return ║ course ║        ║        ║        ║       ║        ║        ║       ║        ║  and  ║ audie… ║
║        ║  For   ║ descr… ║        ║        ║        ║       ║        ║        ║       ║        ║ conc… ║  for   ║
║        ║ Secur… ║  for   ║        ║        ║        ║       ║        ║        ║       ║        ║ stud… ║  the   ║
║        ║        ║ 'Fina… ║        ║        ║        ║       ║        ║        ║       ║        ║ will  ║ course ║
║        ║        ║ Manag… ║        ║        ║        ║       ║        ║        ║       ║        ║ learn ║ 'Fina… ║
║        ║        ║  Risk  ║        ║        ║        ║       ║        ║        ║       ║        ║ in a  ║ Manag… ║
║        ║        ║  and   ║        ║        ║        ║       ║        ║        ║       ║        ║ cour… ║  Risk  ║
║        ║        ║ Return ║        ║        ║        ║       ║        ║        ║       ║        ║ titl… ║  and   ║
║        ║        ║  For   ║        ║        ║        ║       ║        ║        ║       ║        ║ 'Fin… ║ Return ║
║        ║        ║ Secur… ║        ║        ║        ║       ║        ║        ║       ║        ║ Mana… ║  For   ║
║        ║        ║ simil… ║        ║        ║        ║       ║        ║        ║       ║        ║ Risk  ║ Secur… ║
║        ║        ║   to   ║        ║        ║        ║       ║        ║        ║       ║        ║  and  ║  The   ║
║        ║        ║ cours… ║        ║        ║        ║       ║        ║        ║       ║        ║ Retu… ║ audie… ║
║        ║        ║   on   ║        ║        ║        ║       ║        ║        ║       ║        ║  For  ║ descr… ║
║        ║        ║ Udemy. ║        ║        ║        ║       ║        ║        ║       ║        ║ Secu… ║ should ║
║        ║        ║  The   ║        ║        ║        ║       ║        ║        ║       ║        ║ on an ║   be   ║
║        ║        ║ descr… ║        ║        ║        ║       ║        ║        ║       ║        ║ onli… ║ clear, ║
║        ║        ║ should ║        ║        ║        ║       ║        ║        ║       ║        ║ plat… ║ engag… ║
║        ║        ║ empha… ║        ║        ║        ║       ║        ║        ║       ║        ║ like  ║  and   ║
║        ║        ║  the   ║        ║        ║        ║       ║        ║        ║       ║        ║ Udem… ║ indic… ║
║        ║        ║  key   ║        ║        ║        ║       ║        ║        ║       ║        ║  The  ║  the   ║
║        ║        ║ benef… ║        ║        ║        ║       ║        ║        ║       ║        ║ skil… ║ backg… ║
║        ║        ║ pract… ║        ║        ║        ║       ║        ║        ║       ║        ║ shou… ║   or   ║
║        ║        ║ appli… ║        ║        ║        ║       ║        ║        ║       ║        ║  be   ║ exper… ║
║        ║        ║  and   ║        ║        ║        ║       ║        ║        ║       ║        ║ acti… ║ needed ║
║        ║        ║ provi… ║        ║        ║        ║       ║        ║        ║       ║        ║  and  ║  for   ║
║        ║        ║   a    ║        ║        ║        ║  

## **3.2 Insert Data into Users Table**

In [38]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.9 MB/s eta 0:00:00


In [39]:
from faker import Faker
import json
from rich.progress import track

In [40]:
fake = Faker()

In [45]:
def generate_fake_users(num_users=150):
    """
    📌 Generates fake user data with realistic attributes.

    ✅ Uses Faker for realistic full names, emails, and bios.
    ✅ Randomly assigns roles, years of experience, and project availability.
    ✅ Converts JSON-compatible fields correctly.
    ✅ Uses default values for missing columns.
    """
    users = []
    roles = ["student", "instructor", "admin"]
    learning_platforms = ["Udemy", "Coursera", "edX", "Kaggle", "Pluralsight"]

    for _ in range(num_users):
        user = {
            "full_name": fake.name(),
            "email": fake.email(),
            "password_hash": fake.sha256(),
            "role": random.choice(roles),
            "bio": fake.sentence(),
            "linkedin_url": fake.url(),
            "github_url": fake.url(),
            "profile_picture": fake.image_url(),
            "preferences": json.dumps({"theme": random.choice(["light", "dark"]), "notifications": random.choice([True, False])}),
            "education_level": random.choice(["Bachelor's", "Master's", "PhD"]),
            "years_of_experience": random.randint(0, 20),
            "upload_cv_url": fake.url(),
            "preferred_learning_platform": random.choice(learning_platforms),
            "location": fake.city(),
            "available_for_projects": random.choice([True, False]),
            "programming_languages": ", ".join(fake.words(nb=3, ext_word_list=["Python", "C++", "Java", "JavaScript", "SQL", "Swift", "Go", "Rust"])),
            "tools_and_technologies": ", ".join(fake.words(nb=3, ext_word_list=["TensorFlow", "PyTorch", "Docker", "Kubernetes", "Git", "JIRA"])),
            "interests": ", ".join(fake.words(nb=3, ext_word_list=["AI", "Web Development", "Cybersecurity", "Embedded Systems"])),
            "preferred_project_types": ", ".join(fake.words(nb=2, ext_word_list=["Freelance", "Startup", "Enterprise", "Research"])),
            "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "updated_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        users.append(user)

    return users

In [46]:
def insert_fake_users(db_path, table_name, num_users=150):
    """
    📌 Inserts fake user data into a MySQL database.

    ✅ Deletes existing records before inserting new ones.
    ✅ Uses UUIDs for user_IDs.
    ✅ Handles JSON and Boolean data properly.
    """

    try:
        # 🎯 Connect to the MySQL database
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # 🔍 Check if the table exists in MySQL
        table_check_query = f"""
            SELECT COUNT(*)
            FROM information_schema.tables
            WHERE table_schema = '{DB_NAME}'
            AND table_name = '{table_name}';
        """
        cursor.execute(table_check_query)
        table_exists = cursor.fetchone()[0]

        if not table_exists:
            console.print(f"[bold red]❌ Table '{table_name}' does not exist![/bold red]")
            return

        # 🛠 Delete existing records
        delete_query = f"DELETE FROM {table_name};"
        cursor.execute(delete_query)
        conn.commit()
        console.print(f"[bold yellow]⚠️ Deleted existing records from '{table_name}'.[/bold yellow]")

        # 📌 Generate Fake User Data
        users = generate_fake_users(num_users)

        # 🛠 Insert data into the table
        console.print("[bold cyan]⏳ Inserting fake users into the database...[/bold cyan]")
        insert_query = f"""
            INSERT INTO {table_name} (
                user_ID, full_name, email, password_hash, role, bio, linkedin_url, github_url,
                profile_picture, preferences, education_level, years_of_experience, upload_cv_url,
                preferred_learning_platform, location, available_for_projects, programming_languages,
                tools_and_technologies, interests, preferred_project_types, created_at, updated_at
            ) VALUES (UUID(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """

        for user in tqdm(users, total=num_users, desc="Inserting Users", unit="user", leave=False, dynamic_ncols=True):
            cursor.execute(insert_query, tuple(user.values()))

        conn.commit()
        console.print(f"[bold green]✅ Successfully inserted {num_users} fake users into '{table_name}'![/bold green] 🎉")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

In [47]:
insert_fake_users(DB_NAME, "users", 10)

⚠️ Deleted existing records from 'users'.

⏳ Inserting fake users into the database...

✅ Successfully inserted 10 fake users into 'users'! 🎉

In [48]:
display_table_sample("users", 5)

                                            📊 Sample Data from 'users'                                            
╔════╦════╦════╦════╦═══╦════╦═══╦════╦═══╦════╦═══╦════╦═══╦════╦═══╦════╦═══╦════╦═══╦════╦═══╦════╦═══╦════╦═══╗
║ u… ║ f… ║ e… ║ p… ║ … ║ b… ║ … ║ g… ║ … ║ p… ║ … ║ y… ║ … ║ p… ║ … ║ a… ║ … ║ t… ║ … ║ p… ║ … ║ u… ║ … ║ v… ║ … ║
╠════╬════╬════╬════╬═══╬════╬═══╬════╬═══╬════╬═══╬════╬═══╬════╬═══╬════╬═══╬════╬═══╬════╬═══╬════╬═══╬════╬═══╣
║ e… ║ A… ║ e… ║ d… ║ … ║ P… ║ … ║ h… ║ … ║ {… ║ … ║ 13 ║ … ║ P… ║ … ║ 0  ║ … ║ J… ║ … ║ F… ║ … ║ 2… ║ … ║ 0  ║ … ║
║    ║ F… ║    ║    ║   ║ s… ║   ║    ║   ║ "… ║   ║    ║   ║    ║ … ║    ║ … ║ T… ║ … ║ S… ║ … ║ 1… ║   ║    ║   ║
║    ║    ║    ║    ║   ║ r… ║   ║    ║   ║ "… ║   ║    ║   ║    ║   ║    ║ … ║ P… ║ … ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ p… ║   ║    ║   ║ t… ║   ║    ║   ║    ║   ║    ║   ║    ║ … ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ m… ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ l… ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║
║ e… ║ G… ║ e… ║ 8… ║ … ║ D… ║ … ║ h… ║ … ║ {… ║ … ║ 4  ║ … ║ K… ║ … ║ 0  ║ … ║ J… ║ … ║ E… ║ … ║ 2… ║ … ║ 0  ║ … ║
║    ║ J… ║    ║    ║   ║ t… ║   ║    ║   ║ "… ║   ║    ║   ║    ║ … ║    ║ … ║ P… ║ … ║ E… ║ … ║ 1… ║   ║    ║   ║
║    ║    ║    ║    ║   ║ f… ║   ║    ║   ║ "… ║   ║    ║   ║    ║   ║    ║ … ║ P… ║ … ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ t… ║   ║    ║   ║ f… ║   ║    ║   ║    ║   ║    ║   ║    ║ … ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ g… ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ s… ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║
║ e… ║ G… ║ j… ║ 9… ║ … ║ A… ║ … ║ h… ║ … ║ {… ║ … ║ 10 ║ … ║ C… ║ … ║ 1  ║ … ║ J… ║ … ║ F… ║ … ║ 2… ║ … ║ 0  ║ … ║
║    ║ S… ║    ║    ║   ║ r… ║   ║    ║   ║ "… ║   ║    ║   ║    ║   ║    ║ … ║ D… ║ … ║ R… ║ … ║ 1… ║   ║    ║   ║
║    ║    ║    ║    ║   ║ s… ║   ║    ║   ║ "… ║   ║    ║   ║    ║   ║    ║ … ║ P… ║ … ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ m… ║   ║    ║   ║ f… ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ e… ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ b… ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ a… ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ m… ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║
║ e… ║ C… ║ r… ║ f… ║ … ║ A… ║ … ║ h… ║ … ║ {… ║ … ║ 9  ║ … ║ K… ║ … ║ 1  ║ … ║ K… ║ … ║ S… ║ … ║ 2… ║ … ║ 0  ║ … ║
║    ║ L… ║    ║    ║   ║ v… ║   ║    ║   ║ "… ║   ║    ║   ║    ║   ║    ║ … ║ D… ║ … ║ F… ║ … ║ 1… ║   ║    ║   ║
║    ║    ║    ║    ║   ║ c… ║   ║    ║   ║ "… ║   ║    ║   ║    ║   ║    ║ … ║ G… ║ … ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ s… ║   ║    ║   ║ t… ║   ║    ║   ║    ║   ║    ║   ║    ║ … ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ w… ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ p… ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ c… ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║    ║   ║
║ e… ║ S… ║ w… ║ 3… ║ … ║ R… ║ … ║ h… ║ … ║ {… ║ … ║ 2  ║ … ║ U… ║ … ║ 1  ║ … ║ K… ║ … ║ R… ║ … ║ 2… ║ … ║ 0  ║ … ║
║    ║ L… ║    ║    ║   ║ r… ║   ║    ║   ║ "… ║   ║    ║   ║    ║ … ║    ║ … ║ T… ║ … ║ S… ║ … ║ 1… ║   ║    ║   ║
║    ║    ║    ║    ║   ║ b… ║   ║    ║   ║ "… ║   ║    ║   ║    ║   ║    ║ … ║ T… ║ … ║    ║   ║    ║   ║    ║   ║
║    ║    ║    ║    ║   ║ c… ║   ║    ║   ║ f… ║   ║    ║

## **3.3 Insert Data into chapters and course_chapters Table**

In [51]:
def generate_fake_chapters(num_chapters=50):
    """📌 Generates fake chapter data."""
    chapters = []
    for _ in range(num_chapters):
        chapter = {
            "title": fake.sentence(nb_words=4),  # Random title for the chapter
            "description": fake.text(max_nb_chars=500),  # Random description
            "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Current timestamp
        }
        chapters.append(chapter)
    return chapters

In [52]:
def generate_fake_course_chapters(course_ids, chapter_ids, num_chapters_per_course=5):
    """📌 Generates fake course-chapter associations."""
    course_chapters = []
    for course_id in course_ids:
        num_chapters = random.randint(1, num_chapters_per_course)
        for position in range(1, num_chapters + 1):
            chapter_id = random.choice(chapter_ids)  # Randomly select a chapter_id
            course_chapters.append((course_id, chapter_id, position))
    return course_chapters

In [53]:
def get_all_course_ids(cursor):
    """📌 Fetches all existing course IDs from the database."""
    cursor.execute("SELECT course_ID FROM courses;")
    result = cursor.fetchall()
    course_ids = [row[0] for row in result]
    return course_ids

In [54]:
def insert_fake_chapters_and_course_chapters(db_path, chapters_table_name, course_chapters_table_name, num_chapters=50, num_chapters_per_course=5):
    """📌 Inserts fake chapter and course_chapter data into MySQL database."""
    console = Console()

    try:
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # 🛠 Delete existing records
        cursor.execute(f"DELETE FROM {chapters_table_name};")
        cursor.execute(f"DELETE FROM {course_chapters_table_name};")
        conn.commit()
        console.print(f"[bold yellow]⚠️ Deleted existing records from '{chapters_table_name}' and '{course_chapters_table_name}'.[/bold yellow]")

        # 📌 Fetch all course IDs from the database
        course_ids = get_all_course_ids(cursor)

        if not course_ids:
            console.print("[bold red]❌ No courses found in the database![/bold red]")
            return

        # 📌 Generate Fake Chapter Data
        chapters = generate_fake_chapters(num_chapters)

        # 🛠 Insert data into the chapters table
        console.print("[bold cyan]⏳ Inserting fake chapters into the database...[/bold cyan]")
        insert_chapter_query = f"""
            INSERT INTO {chapters_table_name} (title, description, created_at)
            VALUES (%s, %s, %s);
        """

        for chapter in tqdm(chapters, total=num_chapters, desc="Inserting Chapters", unit="chapter", leave=False, dynamic_ncols=True):
            cursor.execute(insert_chapter_query, tuple(chapter.values()))

        # 📌 Get the chapter IDs after insertion
        cursor.execute(f"SELECT chapter_ID FROM {chapters_table_name};")
        chapter_ids = [row[0] for row in cursor.fetchall()]

        # 📌 Generate Fake Course-Chapter Data
        course_chapters = generate_fake_course_chapters(course_ids, chapter_ids, num_chapters_per_course)

        # 🛠 Insert data into the course_chapters table
        console.print("[bold cyan]⏳ Inserting fake course-chapter associations into the database...[/bold cyan]")
        insert_course_chapter_query = f"""
            INSERT INTO {course_chapters_table_name} (course_ID, chapter_ID, position)
            VALUES (%s, %s, %s);
        """

        # To avoid duplicates, we keep track of the inserted pairs
        existing_pairs = set()

        for course_chapter in tqdm(course_chapters, total=len(course_chapters), desc="Inserting Course-Chapter Associations", unit="association", leave=False, dynamic_ncols=True):
            course_id, chapter_id, position = course_chapter

            # Check if the combination of course_id and chapter_id already exists
            if (course_id, chapter_id) not in existing_pairs:
                cursor.execute(insert_course_chapter_query, course_chapter)
                existing_pairs.add((course_id, chapter_id))

        conn.commit()
        console.print(f"[bold green]✅ Successfully inserted {num_chapters} fake chapters and {len(existing_pairs)} unique course-chapter associations into the database![/bold green] 🎉")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

In [55]:
insert_fake_chapters_and_course_chapters(DB_NAME, "chapters", "course_chapters", 5, 5)

⚠️ Deleted existing records from 'chapters' and 'course_chapters'.

⏳ Inserting fake chapters into the database...

⏳ Inserting fake course-chapter associations into the database...

✅ Successfully inserted 5 fake chapters and 39 unique course-chapter associations into the database! 🎉

In [56]:
display_table_sample("chapters", 5)

                                          📊 Sample Data from 'chapters'                                           
╔════════════╦═══════════════════════════════════════╦══════════════════════════════════════╦═════════════════════╗
║ chapter_ID ║                 title                 ║             description              ║     created_at      ║
╠════════════╬═══════════════════════════════════════╬══════════════════════════════════════╬═════════════════════╣
║     62     ║              Nation tax.              ║  Low represent foot focus now work   ║ 2025-04-05 15:04:28 ║
║            ║                                       ║ bar different. According issue role. ║                     ║
║            ║                                       ║  Machine structure buy teacher give  ║                     ║
║            ║                                       ║ although me. Son activity candidate  ║                     ║
║            ║                                       ║   music. Consider second physical    ║                     ║
║            ║                                       ║      appear first free society.      ║                     ║
║            ║                                       ║  Recently hit get middle government  ║                     ║
║            ║                                       ║ maintain. Up bar gas she sing. Group ║                     ║
║            ║                                       ║           ever talk stand.           ║                     ║
║            ║                                       ║ Term myself memory well their drug.  ║                     ║
║            ║                                       ║   Pass culture site buy loss store   ║                     ║
║            ║                                       ║      happy. Create away issue.       ║                     ║
║            ║                                       ║  Share speech decade ten yet future  ║                     ║
║            ║                                       ║   response maybe. During talk fear   ║                     ║
║            ║                                       ║ pick writer building. Offer if begin ║                     ║
║            ║                                       ║                start.                ║                     ║
║     65     ║       Break nice parent choice.       ║ It it serious believe rather memory  ║ 2025-04-05 15:04:28 ║
║            ║                                       ║ process past. Short that line radio  ║                     ║
║            ║                                       ║            indeed store.             ║                     ║
║            ║                                       ║  Meeting yourself situation surface  ║                     ║
║            ║                                       ║        subject little sound.         ║                     ║
║            ║                                       ║   Effort many understand hear time   ║                     ║
║            ║                                       ║             office will.             ║                     ║
║            ║                                       ║  Card laugh goal kind. Stock quite   ║                     ║
║            ║                                       ║ interest everything toward situation ║                     ║
║            ║                                       ║                 we.                  ║                     ║
║            ║                                       ║     Sport teacher your important     ║                     ║
║            ║                                       ║               article.               ║                     ║
║            ║                                       ║  Strong environmental prepare year.  ║                     ║
║            ║                                       ║ Lay indeed produce low election who  ║                     ║
║            ║                                       ║   

In [57]:
display_table_sample("course_chapters", 5)

           📊 Sample Data from 'course_chapters'            
╔══════════════════════════════════╦════════════╦══════════╗
║            course_ID             ║ chapter_ID ║ position ║
╠══════════════════════════════════╬════════════╬══════════╣
║ 8ad6084f74ed8fc5807b5ebae5ccc1ce ║     62     ║    2     ║
║ eafd273ddb0ff0ff841b7ddf4bb1f3ba ║     62     ║    2     ║
║ c611da9088baafd61cc53fcb91f6d1a3 ║     65     ║    5     ║
║ 8ad6084f74ed8fc5807b5ebae5ccc1ce ║     64     ║    1     ║
║ 28ed92e8539afface77410f4cd973d65 ║     61     ║    3     ║
╚══════════════════════════════════╩════════════╩══════════╝

## **3.4 Insert Data into content and chapter_content Table**

In [58]:
def generate_fake_content(num_content=50):
    """📌 Generates fake content data."""
    content_types = ['video', 'pdf', 'quiz', 'ppt', 'text', 'file']
    content = []
    for _ in range(num_content):
        content_item = {
            "content_type": random.choice(content_types),  # Random content type
            "content_url": fake.url(),  # Random URL for the content
            "description": fake.text(max_nb_chars=500),  # Random description
            "duration": round(random.uniform(1, 180), 2),  # Random duration (1 to 180 minutes)
            "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Current timestamp
        }
        content.append(content_item)
    return content

In [59]:
def generate_fake_chapter_content(chapter_ids, content_ids, num_content_per_chapter=5):
    """📌 Generates fake chapter-content associations."""
    chapter_content = []
    for chapter_id in chapter_ids:
        num_contents = random.randint(1, num_content_per_chapter)
        for position in range(1, num_contents + 1):
            content_id = random.choice(content_ids)  # Randomly select a content_id
            chapter_content.append((chapter_id, content_id, position))
    return chapter_content

In [60]:
def get_all_chapter_ids(cursor):
    """📌 Fetches all existing chapter IDs from the database."""
    cursor.execute("SELECT chapter_ID FROM chapters;")
    result = cursor.fetchall()
    chapter_ids = [row[0] for row in result]
    return chapter_ids

In [61]:
def insert_fake_content_and_chapter_content(db_path, content_table_name, chapter_content_table_name, num_content=50, num_content_per_chapter=5):
    """📌 Inserts fake content and chapter_content data into MySQL database."""
    console = Console()

    try:
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # 🛠 Delete existing records
        cursor.execute(f"DELETE FROM {content_table_name};")
        cursor.execute(f"DELETE FROM {chapter_content_table_name};")
        conn.commit()
        console.print(f"[bold yellow]⚠️ Deleted existing records from '{content_table_name}' and '{chapter_content_table_name}'.[/bold yellow]")

        # 📌 Fetch all chapter IDs from the database
        chapter_ids = get_all_chapter_ids(cursor)

        if not chapter_ids:
            console.print("[bold red]❌ No chapters found in the database![/bold red]")
            return

        # 📌 Generate Fake Content Data
        content = generate_fake_content(num_content)

        # Ensure unique URLs
        existing_urls = set()
        unique_content = []
        for content_item in content:
            # Ensure content_url is unique
            while content_item['content_url'] in existing_urls:
                content_item['content_url'] = generate_unique_url()  # This is a function you will need to define
            existing_urls.add(content_item['content_url'])
            unique_content.append(content_item)

        # 🛠 Insert data into the content table
        console.print("[bold cyan]⏳ Inserting fake content into the database...[/bold cyan]")
        insert_content_query = f"""
            INSERT INTO {content_table_name} (content_type, content_url, description, duration, created_at)
            VALUES (%s, %s, %s, %s, %s);
        """

        for content_item in tqdm(unique_content, total=num_content, desc="Inserting Content", unit="content", leave=False, dynamic_ncols=True):
            cursor.execute(insert_content_query, tuple(content_item.values()))

        # 📌 Get the content IDs after insertion
        cursor.execute(f"SELECT content_ID FROM {content_table_name};")
        content_ids = [row[0] for row in cursor.fetchall()]

        # 📌 Generate Fake Chapter-Content Data
        chapter_content = generate_fake_chapter_content(chapter_ids, content_ids, num_content_per_chapter)

        # 🛠 Insert data into the chapter_content table
        console.print("[bold cyan]⏳ Inserting fake chapter-content associations into the database...[/bold cyan]")
        insert_chapter_content_query = f"""
            INSERT INTO {chapter_content_table_name} (chapter_ID, content_ID, position)
            VALUES (%s, %s, %s);
        """

        # To avoid duplicates, we keep track of the inserted pairs
        existing_pairs = set()

        for chapter_cont in tqdm(chapter_content, total=len(chapter_content), desc="Inserting Chapter-Content Associations", unit="association", leave=False, dynamic_ncols=True):
            chapter_id, content_id, position = chapter_cont

            # Check if the combination of chapter_id and content_id already exists
            if (chapter_id, content_id) not in existing_pairs:
                cursor.execute(insert_chapter_content_query, chapter_cont)
                existing_pairs.add((chapter_id, content_id))

        conn.commit()
        console.print(f"[bold green]✅ Successfully inserted {num_content} fake content and {len(existing_pairs)} unique chapter-content associations into the database![/bold green] 🎉")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

# Define the function for generating unique URLs
def generate_unique_url():
    return f"http://uniqueurl_{random.randint(1000, 9999)}.com/"


In [62]:
insert_fake_content_and_chapter_content(DB_NAME, "content", "chapter_content", 5, 5)

⚠️ Deleted existing records from 'content' and 'chapter_content'.

⏳ Inserting fake content into the database...

⏳ Inserting fake chapter-content associations into the database...

✅ Successfully inserted 5 fake content and 14 unique chapter-content associations into the database! 🎉

In [63]:
display_table_sample("content", 5)

                                           📊 Sample Data from 'content'                                           
╔════════════╦══════════════╦═════════════════════════╦══════════════════════════╦══════════╦═════════════════════╗
║ content_ID ║ content_type ║       content_url       ║       description        ║ duration ║     created_at      ║
╠════════════╬══════════════╬═════════════════════════╬══════════════════════════╬══════════╬═════════════════════╣
║     61     ║     file     ║   http://palmer.com/    ║   Only share boy that    ║  124.69  ║ 2025-04-05 15:05:07 ║
║            ║              ║                         ║   light thing. Affect    ║          ║                     ║
║            ║              ║                         ║  recognize year choice   ║          ║                     ║
║            ║              ║                         ║      should moment.      ║          ║                     ║
║            ║              ║                         ║ Above hope group section ║          ║                     ║
║            ║              ║                         ║   shake million fish.    ║          ║                     ║
║            ║              ║                         ║   Mission deep design    ║          ║                     ║
║            ║              ║                         ║   seat. Agree compare    ║          ║                     ║
║            ║              ║                         ║ surface listen activity. ║          ║                     ║
║            ║              ║                         ║     Cut almost plan.     ║          ║                     ║
║            ║              ║                         ║    Scientist make Mrs    ║          ║                     ║
║            ║              ║                         ║         reduce.          ║          ║                     ║
║            ║              ║                         ║   Fine cultural people   ║          ║                     ║
║            ║              ║                         ║ financial. Firm machine  ║          ║                     ║
║            ║              ║                         ║   industry authority.    ║          ║                     ║
║            ║              ║                         ║  Source wish mean them   ║          ║                     ║
║            ║              ║                         ║         family.          ║          ║                     ║
║            ║              ║                         ║ People Congress section  ║          ║                     ║
║            ║              ║                         ║     black have rule      ║          ║                     ║
║            ║              ║                         ║   continue. Money but    ║          ║                     ║
║            ║              ║                         ║       write know.        ║          ║                     ║
║     63     ║     pdf      ║ http://www.wallace.net/ ║ Benefit party born them  ║  113.96  ║ 2025-04-05 15:05:07 ║
║            ║              ║                         ║   they chance. Present   ║          ║                     ║
║            ║              ║                         ║ investment more. Across  ║          ║                     ║
║            ║              ║                         ║       good lawyer.       ║          ║                     ║
║            ║              ║                         ║ Might nothing side. Body ║          ║                     ║
║            ║              ║                         ║ know ok. Trial attorney  ║          ║                     ║
║            ║              ║                         ║    today game first.     ║          ║                     ║
║            ║              ║                         ║      Give building       ║          ║                     ║
║            ║              ║                         ║     information true     ║          ║                     ║
║            ║              ║                         ║ t

In [64]:
display_table_sample("chapter_content", 5)

📊 Sample Data from 'chapter_content' 
╔════════════╦════════════╦══════════╗
║ chapter_ID ║ content_ID ║ position ║
╠════════════╬════════════╬══════════╣
║     61     ║     62     ║    1     ║
║     62     ║     63     ║    3     ║
║     64     ║     63     ║    1     ║
║     65     ║     65     ║    4     ║
║     63     ║     62     ║    3     ║
╚════════════╩════════════╩══════════╝

## **3.5 Insert Data into Student Progress Table**

### **3.5.1 Function to get all the samples from a specific Table**

In [65]:
def get_db_connection():
    return mysql.connector.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DB_NAME
    )

In [66]:
def get_existing_data(cursor, table_name, column_name):
    """ Fetch all existing values of a specific column from a table."""
    cursor.execute(f"SELECT {column_name} FROM {table_name};")
    return [row[0] for row in cursor.fetchall()]

### **3.5.2 Function to generate fake student progress with complete_Rate = 0.4**

In [71]:
def generate_student_progress(user_ids, course_ids, chapter_ids, content_ids, completion_rate=0.4):
    """Generate student progress data with controlled completion rate."""
    student_progress = []

    for user_id in user_ids:
        for course_id in course_ids:
            for chapter_id in chapter_ids:
                for content_id in content_ids:
                    completed = random.random() < completion_rate  # Random completion rate
                    student_progress.append((
                        user_id, course_id, chapter_id, content_id, completed,
                        datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    ))

    return student_progress

### **3.5.3 Insert Fake data into student_progress table**

In [68]:
def insert_fake_student_progress():
    """Inserts fake student progress data into MySQL database."""
    console = Console()

    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        # 🛠 Delete existing records
        cursor.execute("DELETE FROM student_progress;")
        conn.commit()
        console.print(f"[bold yellow]⚠️ Deleted existing records from 'student_progress'.[/bold yellow]")

        # 📌 Fetch all IDs
        user_ids = get_existing_data(cursor, "users", "user_ID")
        course_ids = get_existing_data(cursor, "courses", "course_ID")
        chapter_ids = get_existing_data(cursor, "chapters", "chapter_ID")
        content_ids = get_existing_data(cursor, "content", "content_ID")

        # 📌 Check if courses exist in the courses table
        if not course_ids:
            raise ValueError("No courses found in the 'courses' table. Please ensure there are courses in the database.")

        # 📌 Generate fake student progress data
        student_progress = generate_student_progress(user_ids, course_ids, chapter_ids, content_ids)

        # 🛠 Insert data into student_progress table
        insert_progress_query = """
            INSERT INTO student_progress (user_ID, course_ID, chapter_ID, content_ID, completed, last_updated)
            VALUES (%s, %s, %s, %s, %s, %s);
        """

        console.print("[bold cyan]⏳ Inserting fake student progress data...[/bold cyan]")
        for progress in tqdm(student_progress, total=len(student_progress), desc="Inserting Progress", unit="record", leave=False, dynamic_ncols=True):
            cursor.execute(insert_progress_query, progress)

        conn.commit()
        console.print(f"[bold green]✅ Successfully inserted {len(student_progress)} student progress records![/bold green] 🎉")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

In [72]:
insert_fake_student_progress()

⚠️ Deleted existing records from 'student_progress'.

⏳ Inserting fake student progress data...

✅ Successfully inserted 3750 student progress records! 🎉

In [73]:
display_table_sample("student_progress", 5)

                                      📊 Sample Data from 'student_progress'                                       
╔═════════════╦════════════════════╦═══════════════════╦════════════╦════════════╦═══════════╦════════════════════╗
║ progress_ID ║      user_ID       ║     course_ID     ║ chapter_ID ║ content_ID ║ completed ║    last_updated    ║
╠═════════════╬════════════════════╬═══════════════════╬════════════╬════════════╬═══════════╬════════════════════╣
║    27298    ║ eb2848f9-122e-11f… ║ bcd9c9fcf3adc356… ║     63     ║     61     ║     1     ║     2025-04-05     ║
║             ║                    ║                   ║            ║            ║           ║      15:05:48      ║
║    26359    ║ eac95363-122e-11f… ║ eafd273ddb0ff0ff… ║     65     ║     62     ║     0     ║     2025-04-05     ║
║             ║                    ║                   ║            ║            ║           ║      15:05:48      ║
║    28091    ║ eadc1a45-122e-11f… ║ dde8c68d90ca9b36… ║     61     ║     64     ║     0     ║     2025-04-05     ║
║             ║                    ║                   ║            ║            ║           ║      15:05:48      ║
║    28701    ║ eab698b3-122e-11f… ║ 5fa8d9cc5cee46a3… ║     63     ║     64     ║     0     ║     2025-04-05     ║
║             ║                    ║                   ║            ║            ║           ║      15:05:48      ║
║    29266    ║ eaa3d9ee-122e-11f… ║ 8ad6084f74ed8fc5… ║     61     ║     64     ║     1     ║     2025-04-05     ║
║             ║                    ║                   ║            ║            ║           ║      15:05:48      ║
╚═════════════╩════════════════════╩═══════════════════╩════════════╩════════════╩═══════════╩════════════════════╝

## **3.6 Insert Data into Enrollments Table**

In [74]:
from uuid import uuid4

In [75]:
def generate_enrollments(user_ids, course_ids):
    """📌 Generates enrollment data with controlled status distribution."""
    enrollments = []

    for user_id in user_ids:
        for course_id in course_ids:
            # Assigning status based on probability
            status = random.choices(
                ["Unenrolled", "Active", "Completed"], weights=[90, 5, 5]
            )[0]

            # Progress based on status
            if status == "Unenrolled":
                progress = 0.0
            elif status == "Active":
                progress = round(random.uniform(1.0, 99.0), 2)
            else:  # Completed
                progress = 100.0

            enrollments.append((
                str(uuid4()), user_id, course_id, status, progress,
                datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            ))

    return enrollments

In [76]:
def insert_fake_enrollments(db_name, table_name):
    """📌 Inserts fake enrollment data into MySQL database."""
    console = Console()

    try:
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=db_name
        )
        cursor = conn.cursor()

        # 🛠 Delete existing records
        cursor.execute(f"DELETE FROM {table_name};")
        conn.commit()
        console.print(f"[bold yellow]⚠️ Deleted existing records from '{table_name}'.[/bold yellow]")

        # 📌 Fetch all user IDs
        user_ids = get_existing_data(cursor, "users", "user_ID")
        if not user_ids:
            console.print("[bold red]❌ No users found in the database![/bold red]")
            return

        # 📌 Fetch all course IDs
        course_ids = get_existing_data(cursor, "courses", "course_ID")
        if not course_ids:
            console.print("[bold red]❌ No courses found in the database![/bold red]")
            return

        # 📌 Generate fake enrollments
        enrollments = generate_enrollments(user_ids, course_ids)

        # 🛠 Insert data into enrollments table
        console.print("[bold cyan]⏳ Inserting fake enrollments...[/bold cyan]")
        insert_query = f"""
            INSERT INTO {table_name} (enroll_ID, user_ID, course_ID, status, progress, started_at)
            VALUES (%s, %s, %s, %s, %s, %s);
        """

        for enrollment in tqdm(enrollments, total=len(enrollments), desc="Inserting Enrollments", unit="record", leave=False, dynamic_ncols=True):
            cursor.execute(insert_query, enrollment)

        conn.commit()
        console.print(f"[bold green]✅ Successfully inserted {len(enrollments)} enrollments![/bold green] 🎉")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

In [77]:
def insert_fake_enrollments(db_name, table_name):
    """📌 Inserts fake enrollment data into MySQL database."""
    console = Console()

    try:
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=db_name
        )
        cursor = conn.cursor()

        # 🛠 Delete existing records
        cursor.execute(f"DELETE FROM {table_name};")
        conn.commit()
        console.print(f"[bold yellow]⚠️ Deleted existing records from '{table_name}'.[/bold yellow]")

        # 📌 Fetch all user IDs
        user_ids = get_existing_data(cursor, "users", "user_ID")
        if not user_ids:
            console.print("[bold red]❌ No users found in the database![/bold red]")
            return

        # 📌 Fetch all course IDs
        course_ids = get_existing_data(cursor, "courses", "course_ID")
        if not course_ids:
            console.print("[bold red]❌ No courses found in the database![/bold red]")
            return

        # 📌 Generate fake enrollments
        enrollments = generate_enrollments(user_ids, course_ids)

        # 🛠 Insert data into enrollments table
        console.print("[bold cyan]⏳ Inserting fake enrollments...[/bold cyan]")
        insert_query = f"""
            INSERT INTO {table_name} (enroll_ID, user_ID, course_ID, status, progress, started_at)
            VALUES (%s, %s, %s, %s, %s, %s);
        """

        for enrollment in tqdm(enrollments, total=len(enrollments), desc="Inserting Enrollments", unit="record", leave=False, dynamic_ncols=True):
            cursor.execute(insert_query, enrollment)

        conn.commit()
        console.print(f"[bold green]✅ Successfully inserted {len(enrollments)} enrollments![/bold green] 🎉")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

In [78]:
insert_fake_enrollments(DB_NAME, "enrollments")

⚠️ Deleted existing records from 'enrollments'.

⏳ Inserting fake enrollments...

✅ Successfully inserted 150 enrollments! 🎉

In [79]:
display_table_sample("enrollments", 5)

                                         📊 Sample Data from 'enrollments'                                         
╔══════════════════════╦══════════════════════╦═════════════════════╦════════════╦══════════╦═════════════════════╗
║      enroll_ID       ║       user_ID        ║      course_ID      ║   status   ║ progress ║     started_at      ║
╠══════════════════════╬══════════════════════╬═════════════════════╬════════════╬══════════╬═════════════════════╣
║ 56818b84-920b-428d-… ║ eaa3d9ee-122e-11f0-… ║ e4ac80a2a3d629f5a4… ║ Unenrolled ║   0.0    ║ 2025-04-05 15:13:45 ║
║ f6bd8ef1-55bc-4f2b-… ║ eaa3d9ee-122e-11f0-… ║ a17feb0d85cf499328… ║ Unenrolled ║   0.0    ║ 2025-04-05 15:13:45 ║
║ 89d99864-60a9-451b-… ║ eb0263e3-122e-11f0-… ║ e4ac80a2a3d629f5a4… ║ Unenrolled ║   0.0    ║ 2025-04-05 15:13:45 ║
║ 9bdb3ca2-b02e-4745-… ║ eb0263e3-122e-11f0-… ║ 8ef57ad6fd26056f69… ║ Unenrolled ║   0.0    ║ 2025-04-05 15:13:45 ║
║ 7ea981d0-4575-49bf-… ║ eadc1a45-122e-11f0-… ║ 8ad6084f74ed8fc580… ║ Unenrolled ║   0.0    ║ 2025-04-05 15:13:45 ║
╚══════════════════════╩══════════════════════╩═════════════════════╩════════════╩══════════╩═════════════════════╝

In [80]:
display_table_sample("student_progress", 5)

                                      📊 Sample Data from 'student_progress'                                       
╔═════════════╦════════════════════╦═══════════════════╦════════════╦════════════╦═══════════╦════════════════════╗
║ progress_ID ║      user_ID       ║     course_ID     ║ chapter_ID ║ content_ID ║ completed ║    last_updated    ║
╠═════════════╬════════════════════╬═══════════════════╬════════════╬════════════╬═══════════╬════════════════════╣
║    26612    ║ eb0263e3-122e-11f… ║ dde8c68d90ca9b36… ║     65     ║     65     ║     0     ║     2025-04-05     ║
║             ║                    ║                   ║            ║            ║           ║      15:05:48      ║
║    27037    ║ ea90207d-122e-11f… ║ 8ad6084f74ed8fc5… ║     65     ║     65     ║     0     ║     2025-04-05     ║
║             ║                    ║                   ║            ║            ║           ║      15:05:48      ║
║    28998    ║ eab698b3-122e-11f… ║ fbb0a3eb0e0b117d… ║     63     ║     61     ║     0     ║     2025-04-05     ║
║             ║                    ║                   ║            ║            ║           ║      15:05:48      ║
║    29109    ║ eaa3d9ee-122e-11f… ║ 6f914aa7f11daefc… ║     65     ║     62     ║     0     ║     2025-04-05     ║
║             ║                    ║                   ║            ║            ║           ║      15:05:48      ║
║    28698    ║ eab698b3-122e-11f… ║ 5fa8d9cc5cee46a3… ║     63     ║     61     ║     1     ║     2025-04-05     ║
║             ║                    ║                   ║            ║            ║           ║      15:05:48      ║
╚═════════════╩════════════════════╩═══════════════════╩════════════╩════════════╩═══════════╩════════════════════╝

## **3.7 Insert Data into Quizes, Quwstions and Student_Quiz_Progress Table**

In [81]:
def insert_fake_quizzes(cursor, content_ids):
    """📌 Inserts a quiz for each content."""
    console.print("[bold cyan]⏳ Inserting quizzes...[/bold cyan]")

    quiz_data = [(content_id, f"Quiz for Content {content_id}", "This is a sample quiz.", 70.0, 100) for content_id in content_ids]

    insert_query = """
        INSERT INTO quizzes (content_ID, title, description, pass_percentage, total_score)
        VALUES (%s, %s, %s, %s, %s);
    """
    cursor.executemany(insert_query, quiz_data)

In [82]:
def insert_fake_questions(cursor, quiz_ids):
    """📌 Inserts 5 random questions for each quiz."""
    console.print("[bold cyan]⏳ Inserting questions...[/bold cyan]")

    question_types = ['MCQ', 'Short Answer', 'True/False']

    for quiz_id in tqdm(quiz_ids, desc="Generating Questions", unit="quiz", leave=False, dynamic_ncols=True):
        question_data = [
            (quiz_id, f"Question {i+1} for Quiz {quiz_id}", random.choice(question_types), "Correct Answer")
            for i in range(5)
        ]

        insert_query = """
            INSERT INTO questions (quiz_ID, question_text, type, correct_answer)
            VALUES (%s, %s, %s, %s);
        """
        cursor.executemany(insert_query, question_data)

In [83]:
def insert_fake_student_quiz_progress(cursor, student_progress):
    """📌 Inserts student quiz progress based on watched content."""
    console.print("[bold cyan]⏳ Inserting student quiz progress...[/bold cyan]")

    quiz_progress_data = []

    for user_id, quiz_id in tqdm(student_progress, desc="Generating Student Progress", unit="record", leave=False, dynamic_ncols=True):
        score = random.randint(50, 100)
        passed = score >= 70  # 70% is the pass mark

        quiz_progress_data.append((
            user_id, quiz_id, random.randint(1, 3), score, passed, datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        ))

    insert_query = """
        INSERT INTO student_quiz_progress (user_ID, quiz_ID, attempt_number, score, passed, completed_at)
        VALUES (%s, %s, %s, %s, %s, %s);
    """
    cursor.executemany(insert_query, quiz_progress_data)


In [84]:
def populate_quiz_tables():
    """📌 Main function to populate quizzes, questions, and student quiz progress."""
    try:
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # 🛠 Delete old data
        console.print("[bold yellow]⚠️ Clearing old data...[/bold yellow]")
        cursor.execute("DELETE FROM student_quiz_progress;")
        cursor.execute("DELETE FROM questions;")
        cursor.execute("DELETE FROM quizzes;")
        conn.commit()

        # 📌 Fetch required IDs
        content_ids = get_existing_data(cursor, "content", "content_ID")
        user_ids = get_existing_data(cursor, "users", "user_ID")
        student_progress = get_existing_data(cursor, "student_progress", "content_ID")

        if not (content_ids and user_ids and student_progress):
            console.print("[bold red]❌ Missing data! Ensure all related tables have records.[/bold red]")
            return

        # 🛠 Insert quizzes
        insert_fake_quizzes(cursor, content_ids)
        conn.commit()

        # 🛠 Fetch quiz IDs after insertion
        quiz_ids = get_existing_data(cursor, "quizzes", "quiz_ID")

        # 🛠 Insert questions
        insert_fake_questions(cursor, quiz_ids)
        conn.commit()

        # 🛠 Insert student quiz progress
        user_quiz_pairs = [(random.choice(user_ids), random.choice(quiz_ids)) for _ in range(len(student_progress))]
        insert_fake_student_quiz_progress(cursor, user_quiz_pairs)
        conn.commit()

        console.print("[bold green]✅ All quizzes, questions, and student quiz progress inserted successfully! 🎉[/bold green]")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

In [85]:
populate_quiz_tables()

⚠️ Clearing old data...

⏳ Inserting quizzes...

⏳ Inserting questions...

⏳ Inserting student quiz progress...

✅ All quizzes, questions, and student quiz progress inserted successfully! 🎉

In [86]:
display_table_sample("quizzes", 5)

                                           📊 Sample Data from 'quizzes'                                           
╔═════════╦════════════╦═══════════════════╦═══════════════════╦═════════════════╦═════════════╦══════════════════╗
║ quiz_ID ║ content_ID ║       title       ║    description    ║ pass_percentage ║ total_score ║    created_at    ║
╠═════════╬════════════╬═══════════════════╬═══════════════════╬═════════════════╬═════════════╬══════════════════╣
║   51    ║     61     ║ Quiz for Content  ║ This is a sample  ║      70.0       ║     100     ║    2025-04-05    ║
║         ║            ║        61         ║       quiz.       ║                 ║             ║     15:14:18     ║
║   53    ║     63     ║ Quiz for Content  ║ This is a sample  ║      70.0       ║     100     ║    2025-04-05    ║
║         ║            ║        63         ║       quiz.       ║                 ║             ║     15:14:18     ║
║   52    ║     62     ║ Quiz for Content  ║ This is a sample  ║      70.0       ║     100     ║    2025-04-05    ║
║         ║            ║        62         ║       quiz.       ║                 ║             ║     15:14:18     ║
║   54    ║     64     ║ Quiz for Content  ║ This is a sample  ║      70.0       ║     100     ║    2025-04-05    ║
║         ║            ║        64         ║       quiz.       ║                 ║             ║     15:14:18     ║
║   55    ║     65     ║ Quiz for Content  ║ This is a sample  ║      70.0       ║     100     ║    2025-04-05    ║
║         ║            ║        65         ║       quiz.       ║                 ║             ║     15:14:18     ║
╚═════════╩════════════╩═══════════════════╩═══════════════════╩═════════════════╩═════════════╩══════════════════╝

In [87]:
display_table_sample("questions", 5)

                                          📊 Sample Data from 'questions'                                          
╔═════════════════╦═════════╦═════════════════╦══════════════╦════════════════╦═════════════════╦═════════════════╗
║   question_ID   ║ quiz_ID ║  question_text  ║     type     ║ correct_answer ║   created_at    ║   updated_at    ║
╠═════════════════╬═════════╬═════════════════╬══════════════╬════════════════╬═════════════════╬═════════════════╣
║ a5955ad9-1230-… ║   55    ║ Question 2 for  ║ Short Answer ║ Correct Answer ║   2025-04-05    ║   2025-04-05    ║
║                 ║         ║     Quiz 55     ║              ║                ║    15:14:19     ║    15:14:19     ║
║ a56fb4be-1230-… ║   53    ║ Question 2 for  ║  True/False  ║ Correct Answer ║   2025-04-05    ║   2025-04-05    ║
║                 ║         ║     Quiz 53     ║              ║                ║    15:14:19     ║    15:14:19     ║
║ a5955c8e-1230-… ║   55    ║ Question 5 for  ║     MCQ      ║ Correct Answer ║   2025-04-05    ║   2025-04-05    ║
║                 ║         ║     Quiz 55     ║              ║                ║    15:14:19     ║    15:14:19     ║
║ a5828950-1230-… ║   54    ║ Question 5 for  ║     MCQ      ║ Correct Answer ║   2025-04-05    ║   2025-04-05    ║
║                 ║         ║     Quiz 54     ║              ║                ║    15:14:19     ║    15:14:19     ║
║ a55ce144-1230-… ║   52    ║ Question 3 for  ║  True/False  ║ Correct Answer ║   2025-04-05    ║   2025-04-05    ║
║                 ║         ║     Quiz 52     ║              ║                ║    15:14:19     ║    15:14:19     ║
╚═════════════════╩═════════╩═════════════════╩══════════════╩════════════════╩═════════════════╩═════════════════╝

In [88]:
display_table_sample("student_quiz_progress", 5)

                                    📊 Sample Data from 'student_quiz_progress'                                    
╔═════════════╦═════════════════════════════════╦═════════╦════════════════╦═══════╦════════╦═════════════════════╗
║ progress_ID ║             user_ID             ║ quiz_ID ║ attempt_number ║ score ║ passed ║    completed_at     ║
╠═════════════╬═════════════════════════════════╬═════════╬════════════════╬═══════╬════════╬═════════════════════╣
║    28456    ║ eb2848f9-122e-11f0-a1e3-0aa4e5… ║   52    ║       1        ║ 90.0  ║   1    ║ 2025-04-05 15:14:19 ║
║    28485    ║ eb3b0bb5-122e-11f0-a1e3-0aa4e5… ║   52    ║       2        ║ 89.0  ║   1    ║ 2025-04-05 15:14:19 ║
║    26078    ║ ea90207d-122e-11f0-a1e3-0aa4e5… ║   52    ║       2        ║ 93.0  ║   1    ║ 2025-04-05 15:14:19 ║
║    26728    ║ eb0263e3-122e-11f0-a1e3-0aa4e5… ║   53    ║       3        ║ 84.0  ║   1    ║ 2025-04-05 15:14:19 ║
║    26651    ║ eb0263e3-122e-11f0-a1e3-0aa4e5… ║   54    ║       2        ║ 79.0  ║   1    ║ 2025-04-05 15:14:19 ║
╚═════════════╩═════════════════════════════════╩═════════╩════════════════╩═══════╩════════╩═════════════════════╝

## **3.8 Insert Data into Assignments Table**

In [89]:
def insert_fake_assignments(cursor, course_ids):
    """📌 Inserts 3 assignments per course."""
    console.print("[bold cyan]⏳ Inserting assignments...[/bold cyan]")

    assignment_data = []
    now = datetime.now()

    for course_id in tqdm(course_ids, desc="Generating Assignments", unit="course", leave=False, dynamic_ncols=True):
        for i in range(3):
            deadline = now + timedelta(days=random.randint(10, 60))  # Assignment deadlines within 2 months
            pass_grade = random.randint(50, 80)  # Random passing grade between 50 and 80

            assignment_data.append((
                course_id,
                f"Assignment {i+1} for Course {course_id}",
                "This is a sample assignment.",
                deadline.strftime("%Y-%m-%d %H:%M:%S"),
                pass_grade
            ))

    insert_query = """
        INSERT INTO assignments (course_ID, title, description, deadline, pass_grade)
        VALUES (%s, %s, %s, %s, %s);
    """
    cursor.executemany(insert_query, assignment_data)

In [90]:
def populate_assignments():
    """📌 Main function to populate assignments for each course."""
    try:
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # 🛠 Delete old data
        console.print("[bold yellow]⚠️ Clearing old data...[/bold yellow]")
        cursor.execute("DELETE FROM assignments;")
        conn.commit()

        # 📌 Fetch required course IDs
        course_ids = get_existing_data(cursor, "courses", "course_ID")

        if not course_ids:
            console.print("[bold red]❌ No courses found! Ensure the courses table has data.[/bold red]")
            return

        # 🛠 Insert assignments
        insert_fake_assignments(cursor, course_ids)
        conn.commit()

        console.print("[bold green]✅ All assignments inserted successfully! 🎉[/bold green]")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

In [91]:
populate_assignments()

⚠️ Clearing old data...

⏳ Inserting assignments...

✅ All assignments inserted successfully! 🎉

In [92]:
display_table_sample("assignments", 5)

                                         📊 Sample Data from 'assignments'                                         
╔════════════════╦════════════════╦════════════════╦════════════════╦═══════════════╦════════════╦════════════════╗
║ assignment_ID  ║   course_ID    ║     title      ║  description   ║   deadline    ║ pass_grade ║   created_at   ║
╠════════════════╬════════════════╬════════════════╬════════════════╬═══════════════╬════════════╬════════════════╣
║ aee14fe1-1230… ║ c1e30cced8eaa… ║  Assignment 3  ║   This is a    ║  2025-06-02   ║     51     ║   2025-04-05   ║
║                ║                ║   for Course   ║     sample     ║   15:14:34    ║            ║    15:14:35    ║
║                ║                ║ c1e30cced8eaa… ║  assignment.   ║               ║            ║                ║
║ aee15129-1230… ║ dde8c68d90ca9… ║  Assignment 2  ║   This is a    ║  2025-04-23   ║     76     ║   2025-04-05   ║
║                ║                ║   for Course   ║     sample     ║   15:14:34    ║            ║    15:14:35    ║
║                ║                ║ dde8c68d90ca9… ║  assignment.   ║               ║            ║                ║
║ aee14681-1230… ║ 3adebca7a2c1c… ║  Assignment 2  ║   This is a    ║  2025-06-04   ║     77     ║   2025-04-05   ║
║                ║                ║   for Course   ║     sample     ║   15:14:34    ║            ║    15:14:35    ║
║                ║                ║ 3adebca7a2c1c… ║  assignment.   ║               ║            ║                ║
║ aee1557d-1230… ║ c611da9088baa… ║  Assignment 1  ║   This is a    ║  2025-05-12   ║     52     ║   2025-04-05   ║
║                ║                ║   for Course   ║     sample     ║   15:14:34    ║            ║    15:14:35    ║
║                ║                ║ c611da9088baa… ║  assignment.   ║               ║            ║                ║
║ aee145f2-1230… ║ 3adebca7a2c1c… ║  Assignment 1  ║   This is a    ║  2025-05-30   ║     60     ║   2025-04-05   ║
║                ║                ║   for Course   ║     sample     ║   15:14:34    ║            ║    15:14:35    ║
║                ║                ║ 3adebca7a2c1c… ║  assignment.   ║               ║            ║                ║
╚════════════════╩════════════════╩════════════════╩════════════════╩═══════════════╩════════════╩════════════════╝

## **3.9 Insert Data into Submissions Table**

In [93]:
def insert_fake_submissions(cursor, user_ids, assignment_ids):
    """📌 Inserts fake submissions for each user and assignment."""
    console.print("[bold cyan]⏳ Inserting submissions...[/bold cyan]")

    submission_data = []
    now = datetime.now()

    for user_id in tqdm(user_ids, desc="Generating Submissions", unit="user", leave=False, dynamic_ncols=True):
        for assignment_id in random.sample(assignment_ids, min(3, len(assignment_ids))):  # Each user gets 3 submissions

            submission_status = random.choices(
                ["Not Submitted", "Pending", "Failed", "Passed"],
                weights=[20, 30, 20, 30], k=1
            )[0]

            grade = None
            if submission_status == "Passed":
                grade = random.uniform(50, 100)
            elif submission_status == "Failed":
                grade = random.uniform(0, 49)

            attempts = random.randint(1, 3)
            file_url = f"https://fakeuploads.com/{user_id}_{assignment_id}.pdf" if submission_status != "Not Submitted" else None
            submitted_at = now - timedelta(days=random.randint(1, 30)) if submission_status != "Not Submitted" else None

            submission_data.append((
                user_id, assignment_id, file_url, grade, submitted_at, attempts, submission_status
            ))

    insert_query = """
        INSERT INTO submissions (user_ID, assignment_ID, file_url, grade, submitted_at, attempts, status)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
    """
    cursor.executemany(insert_query, submission_data)

In [94]:
def populate_submissions():
    """📌 Main function to populate submissions for each assignment."""
    try:
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # 🛠 Delete old data
        console.print("[bold yellow]⚠️ Clearing old data...[/bold yellow]")
        cursor.execute("DELETE FROM submissions;")
        conn.commit()

        # 📌 Fetch required user and assignment IDs
        user_ids = get_existing_data(cursor, "users", "user_ID")
        assignment_ids = get_existing_data(cursor, "assignments", "assignment_ID")

        if not user_ids or not assignment_ids:
            console.print("[bold red]❌ No users or assignments found! Ensure tables have data.[/bold red]")
            return

        # 🛠 Insert submissions
        insert_fake_submissions(cursor, user_ids, assignment_ids)
        conn.commit()

        console.print("[bold green]✅ All submissions inserted successfully! 🎉[/bold green]")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

In [95]:
populate_submissions()

⚠️ Clearing old data...

⏳ Inserting submissions...

✅ All submissions inserted successfully! 🎉

In [96]:
display_table_sample("submissions ", 5)

                                        📊 Sample Data from 'submissions '                                         
╔════════════════╦═══════════════╦════════════════╦═══════════════╦═════════╦════════════════╦══════════╦═════════╗
║ submission_ID  ║    user_ID    ║ assignment_ID  ║   file_url    ║  grade  ║  submitted_at  ║ attempts ║ status  ║
╠════════════════╬═══════════════╬════════════════╬═══════════════╬═════════╬════════════════╬══════════╬═════════╣
║ b50e64c7-1230… ║ eadc1a45-122… ║ aee1521d-1230… ║ https://fake… ║   nan   ║   2025-03-31   ║    2     ║ Pending ║
║                ║               ║                ║               ║         ║    15:14:45    ║          ║         ║
║ b50e31ca-1230… ║ ea90207d-122… ║ aee152a5-1230… ║ https://fake… ║ 78.5852 ║   2025-03-17   ║    2     ║ Passed  ║
║                ║               ║                ║               ║         ║    15:14:45    ║          ║         ║
║ b50e2ec8-1230… ║ eb0263e3-122… ║ aee15129-1230… ║ https://fake… ║   nan   ║   2025-03-09   ║    2     ║ Pending ║
║                ║               ║                ║               ║         ║    15:14:45    ║          ║         ║
║ b50e32b4-1230… ║ ea90207d-122… ║ aee14705-1230… ║ https://fake… ║   nan   ║   2025-03-09   ║    3     ║ Pending ║
║                ║               ║                ║               ║         ║    15:14:45    ║          ║         ║
║ b50e6821-1230… ║ eb1580d9-122… ║ aee151a3-1230… ║ https://fake… ║   nan   ║   2025-04-03   ║    2     ║ Pending ║
║                ║               ║                ║               ║         ║    15:14:45    ║          ║         ║
╚════════════════╩═══════════════╩════════════════╩═══════════════╩═════════╩════════════════╩══════════╩═════════╝

## **3.10 Insert Data into QA Table**

In [97]:
import uuid

In [98]:
def generate_fake_questions(cursor, student_ids, instructor_ids, course_ids):
    """📌 Inserts fake Q&A data into the database."""
    console.print("[bold cyan]⏳ Generating Q&A data...[/bold cyan]")

    qa_data = []
    now = datetime.now()

    sample_questions = [
        "How do I improve my understanding of this topic?",
        "Can you provide additional resources?",
        "I'm stuck on this concept, can you clarify?",
        "What are the best strategies to solve this problem?",
        "Can you explain this with a real-world example?",
        "How is this applied in industry?",
        "Is there an alternative method to solve this?",
        "What are the common mistakes students make here?",
        "How should I approach this assignment?",
        "Can I get more details about this topic?"
    ]

    sample_answers = [
        "Great question! I recommend checking out the course materials in section 3.",
        "This concept is best understood through examples. Let me share one...",
        "That's a common issue. Try focusing on the key formula and its application.",
        "You can practice with problems from [resource] to strengthen your understanding.",
        "This is widely used in industry, especially in [specific application].",
        "An alternative method is [explanation]. Both approaches work well.",
        "Avoid [common mistake] and focus on [key strategy].",
        "Check the assignment guidelines, and let me know if you need further clarification."
    ]

    for student_id in tqdm(student_ids, desc="Inserting Questions", unit="student", leave=False, dynamic_ncols=True):
        for course_id in random.sample(course_ids, min(2, len(course_ids))):  # Each student asks in 2 courses
            question_id = str(uuid.uuid4())
            question_text = random.choice(sample_questions)
            created_at = now - timedelta(days=random.randint(1, 30))

            # 50% of questions are answered
            if random.random() < 0.5 and instructor_ids:
                instructor_id = random.choice(instructor_ids)
                answer_text = random.choice(sample_answers)
            else:
                instructor_id = None
                answer_text = None

            qa_data.append((
                question_id, course_id, student_id, instructor_id, question_text, answer_text, created_at
            ))

    insert_query = """
        INSERT INTO qa (question_ID, course_ID, student_ID, instructor_ID, question, answer, created_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
    """
    cursor.executemany(insert_query, qa_data)

In [99]:
def populate_qa():
    """📌 Main function to populate Q&A for each course."""
    try:
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # 🛠 Delete old data
        console.print("[bold yellow]⚠️ Clearing old Q&A data...[/bold yellow]")
        cursor.execute("DELETE FROM qa;")
        conn.commit()

        # 📌 Fetch student, instructor, and course IDs
        student_ids = get_existing_data(cursor, "users", "user_ID")
        instructor_ids = get_existing_data(cursor, "users", "user_ID")  # Assume instructors are in users table
        course_ids = get_existing_data(cursor, "courses", "course_ID")

        if not student_ids or not course_ids:
            console.print("[bold red]❌ No students or courses found! Ensure tables have data.[/bold red]")
            return

        # 🛠 Insert Q&A
        generate_fake_questions(cursor, student_ids, instructor_ids, course_ids)
        conn.commit()

        console.print("[bold green]✅ All Q&A inserted successfully! 🎉[/bold green]")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

In [100]:
populate_qa()

⚠️ Clearing old Q&A data...

⏳ Generating Q&A data...

✅ All Q&A inserted successfully! 🎉

In [101]:
display_table_sample("qa ", 5)

                                             📊 Sample Data from 'qa '                                             
╔═══════════════╦═══════════════╦═══════════════╦═══════════════╦════════════════╦═══════════════╦════════════════╗
║  question_ID  ║   course_ID   ║  student_ID   ║ instructor_ID ║    question    ║    answer     ║   created_at   ║
╠═══════════════╬═══════════════╬═══════════════╬═══════════════╬════════════════╬═══════════════╬════════════════╣
║ 885bd12e-dd0… ║ df2538ae0bd6… ║ ea90207d-122… ║     None      ║  Is there an   ║     None      ║   2025-03-07   ║
║               ║               ║               ║               ║  alternative   ║               ║    15:14:53    ║
║               ║               ║               ║               ║   method to    ║               ║                ║
║               ║               ║               ║               ║  solve this?   ║               ║                ║
║ f2913e73-ca2… ║ df2538ae0bd6… ║ eac95363-122… ║ eb2848f9-122… ║  Is there an   ║ This concept  ║   2025-03-31   ║
║               ║               ║               ║               ║  alternative   ║    is best    ║    15:14:53    ║
║               ║               ║               ║               ║   method to    ║  understood   ║                ║
║               ║               ║               ║               ║  solve this?   ║    through    ║                ║
║               ║               ║               ║               ║                ║ examples. Let ║                ║
║               ║               ║               ║               ║                ║   me share    ║                ║
║               ║               ║               ║               ║                ║    one...     ║                ║
║ d3737fdc-8b4… ║ bcd9c9fcf3ad… ║ eb2848f9-122… ║ eaa3d9ee-122… ║    How do I    ║   That's a    ║   2025-03-10   ║
║               ║               ║               ║               ║   improve my   ║ common issue. ║    15:14:53    ║
║               ║               ║               ║               ║ understanding  ║ Try focusing  ║                ║
║               ║               ║               ║               ║ of this topic? ║  on the key   ║                ║
║               ║               ║               ║               ║                ║  formula and  ║                ║
║               ║               ║               ║               ║                ║      its      ║                ║
║               ║               ║               ║               ║                ║ application.  ║                ║
║ 7355276c-f34… ║ df2538ae0bd6… ║ eadc1a45-122… ║     None      ║    How do I    ║     None      ║   2025-03-06   ║
║               ║               ║               ║               ║   improve my   ║               ║    15:14:53    ║
║               ║               ║               ║               ║ understanding  ║               ║                ║
║               ║               ║               ║               ║ of this topic? ║               ║                ║
║ 82e3d0dd-7e2… ║ 5fa8d9cc5cee… ║ eaa3d9ee-122… ║ eaa3d9ee-122… ║    Can you     ║    This is    ║   2025-03-16   ║
║               ║               ║               ║               ║  explain this  ║  widely used  ║    15:14:53    ║
║               ║               ║               ║               ║     with a     ║ in industry,  ║                ║
║               ║               ║               ║               ║   real-world   ║ especially in ║                ║
║               ║               ║               ║               ║    example?    ║       .       ║                ║
╚═══════════════╩═══════════════╩═══════════════╩═══════════════╩════════════════╩═══════════════╩════════════════╝

## **3.11 Insert Data into Course Reviews and Ratings Table**

In [102]:
def generate_fake_reviews(cursor, user_ids, course_ids):
    """📌 Inserts fake course reviews into the database."""
    console.print("[bold cyan]⏳ Generating course reviews...[/bold cyan]")

    review_data = []
    now = datetime.now()

    sample_reviews = [
        "Excellent course! Learned a lot and the instructor was great.",
        "The content was okay, but I was expecting more examples.",
        "This course is too basic for someone with experience.",
        "I loved this course! It covered everything I needed.",
        "The course was good, but the pacing was too fast.",
        "Not what I expected, but still decent. Could use more real-world examples.",
        "Great course overall, but some topics felt rushed.",
        "This course was very informative and helped me understand the topic better."
    ]

    for user_id in tqdm(user_ids, desc="Inserting Reviews", unit="user", leave=False, dynamic_ncols=True):
        for course_id in random.sample(course_ids, min(3, len(course_ids))):  # Each user reviews 3 courses
            review_id = str(uuid.uuid4())
            rating = round(random.uniform(1.0, 5.0), 1)  # Random rating between 1 and 5
            review_text = random.choice(sample_reviews) if random.random() < 0.8 else None  # 80% chance of review text
            created_at = now - timedelta(days=random.randint(1, 60))

            review_data.append((
                review_id, user_id, course_id, rating, review_text, created_at
            ))

    insert_query = """
        INSERT INTO course_reviews (review_ID, user_ID, course_ID, rating, review, created_at)
        VALUES (%s, %s, %s, %s, %s, %s);
    """
    cursor.executemany(insert_query, review_data)

In [103]:
def populate_course_reviews():
    """📌 Main function to populate course reviews."""
    try:
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # 🛠 Delete old data
        console.print("[bold yellow]⚠️ Clearing old course reviews...[/bold yellow]")
        cursor.execute("DELETE FROM course_reviews;")
        conn.commit()

        # 📌 Fetch user and course IDs
        user_ids = get_existing_data(cursor, "users", "user_ID")
        course_ids = get_existing_data(cursor, "courses", "course_ID")

        if not user_ids or not course_ids:
            console.print("[bold red]❌ No users or courses found! Ensure tables have data.[/bold red]")
            return

        # 🛠 Insert reviews
        generate_fake_reviews(cursor, user_ids, course_ids)
        conn.commit()

        console.print("[bold green]✅ All course reviews inserted successfully! 🎉[/bold green]")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

In [104]:
populate_course_reviews()

⚠️ Clearing old course reviews...

⏳ Generating course reviews...

✅ All course reviews inserted successfully! 🎉

In [105]:
display_table_sample("course_reviews ", 5)

                                       📊 Sample Data from 'course_reviews '                                       
╔════════════════════╦════════════════════╦════════════════════╦════════╦════════════════════╦════════════════════╗
║     review_ID      ║      user_ID       ║     course_ID      ║ rating ║       review       ║     created_at     ║
╠════════════════════╬════════════════════╬════════════════════╬════════╬════════════════════╬════════════════════╣
║ 93d7ef30-505d-424… ║ eb2848f9-122e-11f… ║ 8ef57ad6fd26056f6… ║  4.9   ║    Great course    ║     2025-02-11     ║
║                    ║                    ║                    ║        ║ overall, but some  ║      15:14:59      ║
║                    ║                    ║                    ║        ║    topics felt     ║                    ║
║                    ║                    ║                    ║        ║      rushed.       ║                    ║
║ 7ff7b1f1-fb4e-45e… ║ eaa3d9ee-122e-11f… ║ fbb0a3eb0e0b117d7… ║  1.4   ║        None        ║     2025-03-06     ║
║                    ║                    ║                    ║        ║                    ║      15:14:59      ║
║ 52a9c5e9-041d-406… ║ eb0263e3-122e-11f… ║ 5fa8d9cc5cee46a3b… ║  3.4   ║ This course is too ║     2025-02-16     ║
║                    ║                    ║                    ║        ║ basic for someone  ║      15:14:59      ║
║                    ║                    ║                    ║        ║  with experience.  ║                    ║
║ 4c8e665b-99b0-488… ║ eac95363-122e-11f… ║ 8ef57ad6fd26056f6… ║  3.0   ║    Great course    ║     2025-03-03     ║
║                    ║                    ║                    ║        ║ overall, but some  ║      15:14:59      ║
║                    ║                    ║                    ║        ║    topics felt     ║                    ║
║                    ║                    ║                    ║        ║      rushed.       ║                    ║
║ 66ebbf72-b372-4e0… ║ eab698b3-122e-11f… ║ 28ed92e8539afface… ║  1.6   ║ Excellent course!  ║     2025-02-14     ║
║                    ║                    ║                    ║        ║ Learned a lot and  ║      15:14:59      ║
║                    ║                    ║                    ║        ║ the instructor was ║                    ║
║                    ║                    ║                    ║        ║       great.       ║                    ║
╚════════════════════╩════════════════════╩════════════════════╩════════╩════════════════════╩════════════════════╝

## **3.12 Insert Data into Certificates and Badges Table**

In [106]:
def generate_fake_certificates(cursor, user_ids, course_ids):
    """📌 Inserts fake certificates into the database."""
    console.print("[bold cyan]⏳ Generating certificates...[/bold cyan]")

    certificate_data = []
    now = datetime.now()

    for user_id in tqdm(user_ids, desc="Inserting Certificates", unit="user", leave=False, dynamic_ncols=True):
        for course_id in random.sample(course_ids, min(3, len(course_ids))):  # Each user gets certificates for 3 courses
            certificate_id = str(uuid.uuid4())
            certificate_url = f"https://certificates.example.com/{certificate_id}"  # Random certificate URL
            issued_at = now - timedelta(days=random.randint(1, 60))

            certificate_data.append((
                certificate_id, user_id, course_id, certificate_url, issued_at
            ))

    insert_query = """
        INSERT INTO certificates (certificate_ID, user_ID, course_ID, certificate_url, issued_at)
        VALUES (%s, %s, %s, %s, %s);
    """
    cursor.executemany(insert_query, certificate_data)

In [107]:
def populate_certificates():
    """📌 Main function to populate certificates."""
    try:
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # 🛠 Delete old data
        console.print("[bold yellow]⚠️ Clearing old certificates...[/bold yellow]")
        cursor.execute("DELETE FROM certificates;")
        conn.commit()

        # 📌 Fetch user and course IDs
        user_ids = get_existing_data(cursor, "users", "user_ID")
        course_ids = get_existing_data(cursor, "courses", "course_ID")

        if not user_ids or not course_ids:
            console.print("[bold red]❌ No users or courses found! Ensure tables have data.[/bold red]")
            return

        # 🛠 Insert certificates
        generate_fake_certificates(cursor, user_ids, course_ids)
        conn.commit()

        console.print("[bold green]✅ All certificates inserted successfully! 🎉[/bold green]")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

In [108]:
populate_certificates()

⚠️ Clearing old certificates...

⏳ Generating certificates...

✅ All certificates inserted successfully! 🎉

In [109]:
display_table_sample("certificates ", 5)

                                        📊 Sample Data from 'certificates '                                        
╔══════════════════════╦══════════════════════╦══════════════════════╦══════════════════════╦═════════════════════╗
║    certificate_ID    ║       user_ID        ║      course_ID       ║   certificate_url    ║      issued_at      ║
╠══════════════════════╬══════════════════════╬══════════════════════╬══════════════════════╬═════════════════════╣
║ fe98e55d-252b-4e5e-… ║ eb1580d9-122e-11f0-… ║ a17feb0d85cf499328f… ║ https://certificate… ║ 2025-02-17 15:15:05 ║
║ 606ab4bf-a58f-4cf9-… ║ eac95363-122e-11f0-… ║ c611da9088baafd61cc… ║ https://certificate… ║ 2025-03-16 15:15:05 ║
║ b2e97c3b-fc03-49ca-… ║ eab698b3-122e-11f0-… ║ 8ad6084f74ed8fc5807… ║ https://certificate… ║ 2025-02-09 15:15:05 ║
║ 21c73bd6-4d5a-4275-… ║ eb2848f9-122e-11f0-… ║ 5fa8d9cc5cee46a3b45… ║ https://certificate… ║ 2025-04-02 15:15:05 ║
║ 24398c4c-f107-4930-… ║ eaa3d9ee-122e-11f0-… ║ eafd273ddb0ff0ff841… ║ https://certificate… ║ 2025-03-15 15:15:05 ║
╚══════════════════════╩══════════════════════╩══════════════════════╩══════════════════════╩═════════════════════╝

## **3.13 Insert Data into Payments Table**

In [110]:
def generate_fake_payments(cursor, user_ids, course_ids):
    """📌 Inserts fake payment data into the database."""
    console.print("[bold cyan]⏳ Generating payments...[/bold cyan]")

    payment_data = []
    now = datetime.now()

    subscription_plans = ['Monthly', 'Annual', 'One-Time']
    payment_statuses = ['Success', 'Pending', 'Failed']

    for user_id in tqdm(user_ids, desc="Inserting Payments", unit="user", leave=False, dynamic_ncols=True):
        for course_id in random.sample(course_ids, min(3, len(course_ids))):  # Each user gets payments for 3 courses
            payment_id = str(uuid.uuid4())
            subscription_plan = random.choice(subscription_plans)
            amount = round(random.uniform(10.0, 500.0), 2)  # Random amount between $10 and $500
            payment_status = random.choice(payment_statuses)
            created_at = now - timedelta(days=random.randint(1, 90))  # Random date in the last 90 days

            payment_data.append((
                payment_id, user_id, course_id, subscription_plan, amount, payment_status, created_at
            ))

    insert_query = """
        INSERT INTO payments (payment_ID, user_ID, course_ID, subscription_plan, amount, payment_status, created_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
    """
    cursor.executemany(insert_query, payment_data)

In [111]:
def populate_payments():
    """📌 Main function to populate payments."""
    try:
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DB_NAME
        )
        cursor = conn.cursor()

        # 🛠 Delete old data
        console.print("[bold yellow]⚠️ Clearing old payments...[/bold yellow]")
        cursor.execute("DELETE FROM payments;")
        conn.commit()

        # 📌 Fetch user and course IDs
        user_ids = get_existing_data(cursor, "users", "user_ID")
        course_ids = get_existing_data(cursor, "courses", "course_ID")

        if not user_ids or not course_ids:
            console.print("[bold red]❌ No users or courses found! Ensure tables have data.[/bold red]")
            return

        # 🛠 Insert payments
        generate_fake_payments(cursor, user_ids, course_ids)
        conn.commit()

        console.print("[bold green]✅ All payments inserted successfully! 🎉[/bold green]")

    except Exception as e:
        console.print(f"[bold red]❌ Error:[/bold red] {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()

In [112]:
populate_payments()

⚠️ Clearing old payments...

⏳ Generating payments...

✅ All payments inserted successfully! 🎉

In [113]:
display_table_sample("payments ", 5)

                                          📊 Sample Data from 'payments '                                          
╔═════════════════╦═════════════════╦════════════════╦═════════════════╦════════╦════════════════╦════════════════╗
║   payment_ID    ║     user_ID     ║   course_ID    ║ subscription_p… ║ amount ║ payment_status ║   created_at   ║
╠═════════════════╬═════════════════╬════════════════╬═════════════════╬════════╬════════════════╬════════════════╣
║ 396b67fd-c2f7-… ║ eaa3d9ee-122e-… ║ df2538ae0bd65… ║    One-Time     ║ 474.45 ║     Failed     ║   2025-03-08   ║
║                 ║                 ║                ║                 ║        ║                ║    15:15:10    ║
║ e9399da5-9aaa-… ║ eb0263e3-122e-… ║ eafd273ddb0ff… ║     Monthly     ║ 289.13 ║    Success     ║   2025-03-31   ║
║                 ║                 ║                ║                 ║        ║                ║    15:15:10    ║
║ 95ba9894-82a1-… ║ eadc1a45-122e-… ║ 8ef57ad6fd260… ║     Annual      ║ 162.43 ║     Failed     ║   2025-01-27   ║
║                 ║                 ║                ║                 ║        ║                ║    15:15:10    ║
║ e45dd1f1-0772-… ║ eb3b0bb5-122e-… ║ 3adebca7a2c1c… ║     Annual      ║ 351.34 ║    Pending     ║   2025-03-12   ║
║                 ║                 ║                ║                 ║        ║                ║    15:15:10    ║
║ 0138f2c5-6b61-… ║ ea90207d-122e-… ║ 3adebca7a2c1c… ║     Monthly     ║ 373.97 ║    Pending     ║   2025-02-13   ║
║                 ║                 ║                ║                 ║        ║                ║    15:15:10    ║
╚═════════════════╩═════════════════╩════════════════╩═════════════════╩════════╩════════════════╩════════════════╝

# Checking the database after finnishing

In [114]:
inspect_database(DB_NAME, HOST, USER, PASSWORD)

✅ Successfully connected to the MySQL database!

🛠️  Inspecting table: assignments

 Attribute Name   Data Type  
 ──────────────────────────── 
  assignment_ID    char(36)   
  course_ID        char(36)   
  title            text       
  description      text       
  deadline         datetime   
  pass_grade       int        
  created_at       datetime  

📦  The Table has dataassignments:

🛠️  Inspecting table: certificates

...

🛠️  Inspecting table: users

 Attribute Name                Data Type     
 ──────────────────────────────────────────── 
  user_ID                       char(36)      
  full_name                     text          
  email                         varchar(255)  
  password_hash                 text          
  role                          text          
  bio                           text          
  linkedin_url                  text          
  github_url                    text          
  profile_picture               text          
  preferences                   json          
  education_level               text          
  years_of_experience           int           
  upload_cv_url                 text          
  preferred_learning_platform   text          
  location                      text          
  available_for_projects        tinyint(1)    
  programming_languages         text          
  tools_and_technologies        text          
  interests                     text          
  preferred_project_types       text          
  created_at                    timestamp     
  updated_at                    timestamp     
  email_verification_token      varchar(255)  
  verified                      tinyint(1)    
  email_verification_expires    datetime     

📦  The Table has datausers:

📊  Database Summary:

╭───────────────────────┬──────────────────────┬────────────────╮
│ Table Name            │ Number of Attributes │ Number of Rows │
├───────────────────────┼──────────────────────┼────────────────┤
│ assignments           │ 7                    │ 45             │
│ certificates          │ 5                    │ 30             │
│ chapter_content       │ 3                    │ 14             │
│ chapters              │ 4                    │ 5              │
│ content               │ 6                    │ 5              │
│ course_chapters       │ 3                    │ 39             │
│ course_reviews        │ 6                    │ 30             │
│ courses               │ 13                   │ 15             │
│ enrollments           │ 6                    │ 150            │
│ interactions          │ 5                    │ 0              │
│ password_resets       │ 5                    │ 0              │
│ payments              │ 7                    │ 30             │
│ qa                    │ 7                    │ 20             │
│ questions             │ 7                    │ 25             │
│ quizzes               │ 7                    │ 5              │
│ student_progress      │ 7                    │ 3750           │
│ student_quiz_progress │ 7                    │ 3750           │
│ submissions           │ 8                    │ 30             │
│ users                 │ 25                   │ 10             │
╰───────────────────────┴──────────────────────┴────────────────╯

# Save Backup for the database

In [115]:
import os
import subprocess
import pickle
import google.auth
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from datetime import datetime

In [116]:
# Backup table data and save as SQL files
def backup_table_to_sql(table_name, conn, cursor, backup_dir):
    cursor.execute(f"SHOW CREATE TABLE {table_name}")
    create_table_sql = cursor.fetchone()[1]

    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    table_backup = os.path.join(backup_dir, f"{table_name}.sql")
    with open(table_backup, 'w') as file:
        file.write(f"DROP TABLE IF EXISTS {table_name};\n")
        file.write(create_table_sql + ";\n")
        for row in rows:
            file.write(f"INSERT INTO {table_name} VALUES {str(row)};\n")

    print(f"Backup of table {table_name} saved as {table_backup}")

# Specify the Google Drive path for backup
backup_dir = "/Database_Backups/"

# Create backup directory if it doesn't exist
os.makedirs(backup_dir, exist_ok=True)

# List of tables you want to backup
tables = ["assignments", "certificates", "chapter_content", "chapters", "content"]  # Add all your tables here

# Backup each table
for table in tables:
    backup_table_to_sql(table, db_conn, cursor, backup_dir)

cursor.close()
db_conn.close()

Backup of table assignments saved as /Database_Backups/assignments.sql
Backup of table certificates saved as /Database_Backups/certificates.sql
Backup of table chapter_content saved as /Database_Backups/chapter_content.sql
Backup of table chapters saved as /Database_Backups/chapters.sql
Backup of table content saved as /Database_Backups/content.sql


In [117]:
!pip install pymysql
!pip install colorama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.4 MB/s eta 0:00:00


In [118]:
import os
import datetime
import pymysql
from colorama import Fore, Style, init

In [119]:
init(autoreset=True)

def connect_to_mysql(host, user, password, db=None):
    return pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=db,
        charset='utf8mb4',
        cursorclass=pymysql.cursors.Cursor
    )

In [120]:
def backup_database(host, user, password, database):
    print(f"{Fore.CYAN}🔐 Connecting to MySQL server...")

    try:
        conn = connect_to_mysql(host, user, password, db=database)
        cursor = conn.cursor()
        print(f"{Fore.GREEN}✅ Connected to database: {database}\n")
    except Exception as e:
        print(f"{Fore.RED}❌ Connection failed: {e}")
        return

    # Create backup directory and filename
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_dir = os.path.join(os.getcwd(), "Database_Backups")
    os.makedirs(backup_dir, exist_ok=True)
    backup_file = os.path.join(backup_dir, f"{database}_backup_{timestamp}.sql")

    try:
        with open(backup_file, 'w', encoding='utf-8') as f:
            f.write(f"-- Full backup of `{database}` created on {timestamp}\n\n")
            f.write(f"CREATE DATABASE IF NOT EXISTS `{database}`;\nUSE `{database}`;\n\n")

            cursor.execute("SHOW TABLES")
            tables = [row[0] for row in cursor.fetchall()]

            for table in tables:
                print(f"{Fore.BLUE}⏳ Backing up table: {table}...")
                cursor.execute(f"SHOW CREATE TABLE `{table}`")
                create_sql = cursor.fetchone()[1]

                cursor.execute(f"SELECT * FROM `{table}`")
                rows = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]

                f.write(f"\n-- ------------------------------\n")
                f.write(f"-- Structure for table `{table}`\n")
                f.write(f"-- ------------------------------\n")
                f.write(f"DROP TABLE IF EXISTS `{table}`;\n")
                f.write(create_sql + ";\n\n")

                f.write(f"-- ------------------------------\n")
                f.write(f"-- Data for table `{table}`\n")
                f.write(f"-- ------------------------------\n")

                for row in rows:
                    escaped_values = []
                    for value in row:
                        if value is None:
                            escaped_values.append("NULL")
                        else:
                            value_str = str(value).replace("'", "''")
                            escaped_values.append(f"'{value_str}'")
                    values_str = ", ".join(escaped_values)
                    columns_str = ", ".join(f"`{col}`" for col in columns)
                    f.write(f"INSERT INTO `{table}` ({columns_str}) VALUES ({values_str});\n")

                print(f"{Fore.GREEN}✅ Table `{table}` backed up.\n")

        print(f"{Fore.MAGENTA}📦 Full database backup saved: {backup_file}")

    except Exception as e:
        print(f"{Fore.RED}❌ Failed during backup: {e}")

    cursor.close()
    conn.close()

In [122]:
backup_database(HOST, USER, PASSWORD, DB_NAME)

🔐 Connecting to MySQL server...
✅ Connected to database: CourseDataBase

⏳ Backing up table: assignments...
✅ Table `assignments` backed up.

⏳ Backing up table: certificates...
✅ Table `certificates` backed up.

⏳ Backing up table: chapter_content...
✅ Table `chapter_content` backed up.

⏳ Backing up table: chapters...
✅ Table `chapters` backed up.

⏳ Backing up table: content...
✅ Table `content` backed up.

⏳ Backing up table: course_chapters...
✅ Table `course_chapters` backed up.

⏳ Backing up table: course_reviews...
✅ Table `course_reviews` backed up.

⏳ Backing up table: courses...
✅ Table `courses` backed up.

⏳ Backing up table: enrollments...
✅ Table `enrollments` backed up.

⏳ Backing up table: interactions...
✅ Table `interactions` backed up.

⏳ Backing up table: password_resets...
✅ Table `password_resets` backed up.

⏳ Backing up table: payments...
✅ Table `payments` backed up.

⏳ Backing up table: qa...
✅ Table `qa` backed up.

⏳ Backing up table: questions...
✅ Table `

In [123]:
def connect_to_mysql(host, user, password, db):
    """Helper function to establish MySQL connection"""
    try:
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            db=db,
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor
        )
        return connection
    except pymysql.MySQLError as e:
        console.print(f"[red]❌ Error connecting to MySQL: {e}[/red]")
        return None

In [129]:
def backup_database(host, user, password, database):
    console.print("[cyan]🔐 Connecting to MySQL server...[/cyan]")

    try:
        conn = connect_to_mysql(host, user, password, db=database)
        cursor = conn.cursor()
        console.print(f"[green]✅ Connected to database: {database}\n[/green]")
    except Exception as e:
        console.print(f"[red]❌ Connection failed: {e}[/red]")
        return

    # Create the 'database_backup' directory if it doesn't exist
    backup_dir = os.path.join(os.getcwd(), "database_backup")
    os.makedirs(backup_dir, exist_ok=True)

    # Define the backup file name with .db extension
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_file = os.path.join(backup_dir, f"{database}_backup_{timestamp}.db")

    try:
        with open(backup_file, 'w', encoding='utf-8') as f:
            f.write(f"-- Full backup of `{database}` created on {timestamp}\n\n")
            f.write(f"CREATE DATABASE IF NOT EXISTS `{database}`;\nUSE `{database}`;\n\n")

            cursor.execute("SHOW TABLES")
            tables = [row[0] for row in cursor.fetchall()]

            for table in tables:
                console.print(f"[yellow]⏳ Backing up table: {table}...[/yellow]")
                cursor.execute(f"SHOW CREATE TABLE `{table}`")
                create_sql = cursor.fetchone()[1]

                cursor.execute(f"SELECT * FROM `{table}`")
                rows = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]

                f.write(f"\n-- ------------------------------\n")
                f.write(f"-- Structure for table `{table}`\n")
                f.write(f"-- ------------------------------\n")
                f.write(f"DROP TABLE IF EXISTS `{table}`;\n")
                f.write(create_sql + ";\n\n")

                f.write(f"-- ------------------------------\n")
                f.write(f"-- Data for table `{table}`\n")
                f.write(f"-- ------------------------------\n")

                for row in rows:
                    escaped_values = []
                    for value in row:
                        if value is None:
                            escaped_values.append("NULL")
                        else:
                            value_str = str(value).replace("'", "''")
                            escaped_values.append(f"'{value_str}'")
                    values_str = ", ".join(escaped_values)
                    columns_str = ", ".join(f"`{col}`" for col in columns)
                    f.write(f"INSERT INTO `{table}` ({columns_str}) VALUES ({values_str});\n")

                console.print(f"[green]✅ Table `{table}` backed up.\n[/green]")

        console.print(f"[magenta]📦 Full database backup saved: {backup_file}\n[/magenta]")

    except Exception as e:
        console.print(f"[red]❌ Failed during backup: {e}[/red]")

    finally:
        cursor.close()
        conn.close()


In [130]:
backup_database(host=HOST, user=USER, password=PASSWORD, database=DB_NAME)

🔐 Connecting to MySQL server...

✅ Connected to database: CourseDataBase

❌ Failed during backup: 0